In [1]:
import csv
import pandas as pd
import math
import json
import numpy as np
import random
import matplotlib as plt
import re

### wangshuo的原始欺诈数据，只到2019年，没有2020和2021年

In [194]:
total = pd.read_csv('wangshuo/fraud2.csv',encoding="utf-8",header=None)
print(len(set(total.iloc[:,1])))
fraud_time = pd.read_csv('wangshuo/fraud-time2.csv',encoding="utf-8",header=None)
print(fraud_time.shape[0])
print(len(set(fraud_time.iloc[:,0])))
# wangshuo欺诈的公司数据，包括股票代码、欺诈年份、行业、总资产
fraud = pd.read_csv('wangshuo/frau_time_industry_money.csv',encoding="utf-8", header=None)
# wangshuo非欺诈的公司数据，包括股票代码、欺诈年份、行业、总资产
nonfraud = pd.read_csv('wangshuo/all_time_industry_money.csv',encoding="utf-8", header=None)
# wangshuo欺诈的公司和非欺诈的公司按照行业一致、时间一致、公司规模基本一致、总资产来1：1配对
match= pd.read_csv('wangshuo/match_time_industry_money.csv',encoding="utf-8", header=None)

1323
1255
1255


### 从现在开始，重新从国泰安下载和处理欺诈数据

In [55]:
weigui = pd.read_csv('违规信息总表/STK_Violation_Main_raw.csv',encoding="utf-8")
print(weigui[''])
# 把违规的时间（ViolationYear）处理成 list 存在原处
for i in range(weigui.shape[0]):
    time = weigui.loc[i,'ViolationYear']
    #print(time)
    if weigui.loc[i,'ViolationYear'] is np.nan:
        continue
    time = time.replace(',','')
    n_year = len(time)/4
    weigui_years = []
    last_year = int(time[:4])
    for y in range(int(n_year)):
        weigui_years.append(last_year+y)
    weigui.loc[i,'ViolationYear'] = json.dumps(weigui_years)
weigui
#weigui.to_csv('违规信息总表/STK_Violation_Main.csv',encoding='gbk',index=None)

,ViolationID,Symbol,DisposalDate,DeclareDate,ViolationType,ViolationYear,IsViolated,ShortName,CoFullName
0,4095400,2,2016/7/21,2016/7/21,重大遗漏,"[2015, 2016]",N,万科A,万科企业股份有限公司
1,4095403,2,2016/7/21,2016/7/21,其他,[2016],Y,万科A,万科企业股份有限公司
2,401009,3,2001/6/8,2001/6/9,推迟披露,[2001],Y,PT 金田A,金田实业(集团)股份有限公司
3,401870,3,2002/3/5,2002/3/5,其他,[2001],Y,PT 金田A,金田实业(集团)股份有限公司
4,401008,3,2002/5/9,2002/5/10,推迟披露,[2002],Y,PT 金田A,金田实业(集团)股份有限公司
5,40108118,3,2002/5/9,2002/5/10,其他,"[2020, 2021]",N,PT 金田A,金田实业(集团)股份有限公司
6,40967,4,2010/1/29,2010/1/30,"虚假记载(误导性陈述),其他","[2008, 2009, 2010]",Y,*ST 国农,深圳中国农大科技股份有限公司
7,401647,4,2010/9/20,2010/9/29,"重大遗漏,披露不实(其它),其他","[2005, 2006, 2007]",N,ST 国农,深圳中国农大科技股份有限公司
8,405974,4,2013/4/12,2013/6/6,"虚假记载(误导性陈述),一般会计处理不当,其他","[2010, 2011, 2012, 2013]",Y,国农科技,深圳中国农大科技股份有限公司
9,4056066,4,2018/8/15,2018/8/16,推迟披露,"[2014, 2015, 2016]",Y,国农科技,深圳中国农大科技股份有限公司


In [109]:
weigui = pd.read_csv('违规信息总表/STK_Violation_Main.csv',encoding="gbk")
print('共有 %d 家违规公司' % len(set(weigui['Symbol'])))
print('共有{}项违规文件'.format(weigui.shape))

共有 2825 家违规公司
共有(11995, 9)项违规文件


In [240]:
industry_dict ={
    # 这四个是CG_CO里没有的
    '金属制品、机械和设备修理业':'工业',
    '其他服务业':'公用事业',
    '居民服务业':'公用事业',
    '管道运输业':'公用事业'
}

CG_CO= pd.read_csv('行业_公司基本情况文件/CG_Co.csv',encoding='utf-8')
for i in range(CG_CO.shape[0]):
    key = CG_CO.loc[i,'Nnindnme']
    value = CG_CO.loc[i,'Indnme']
    industry_dict[key] = value

# 会计信息质量->财务报告信息，包括财务指标、行业、公司规模
# 表格预处理，包括年份的提取，然后行业的进一步分类
basic_info = pd.read_excel('会计信息质量-财务指标/AIQ_LCFinIndexY_raw.xlsx',encoding="gbk")
basic_info = basic_info.drop(index=[0,1],axis=0)
basic_info = basic_info.reset_index(drop = True)

# 对行业做一个更上层的归类
basic_info['Industry'] = [industry_dict[i] for i in basic_info['IndustryName']]

time = basic_info['EndDate']
years = []
for t in time:
    years.append(t.split('-')[0])
basic_info['EndDate'] = years

# 中文编码的大小：gbk<gb2312<gb18030，一般情况下，gb18030编码都能解析gbk不能解析的编码信息。
basic_info.to_excel('会计信息质量-财务指标/AIQ_LCFinIndexY.xlsx',encoding='gb18030',index=None)
# 不存为csv文件是因为csv文件会把11次方以上的数值后面的小数抹去

In [241]:
basic_info = pd.read_excel('会计信息质量-财务指标/AIQ_LCFinIndexY.xlsx',encoding='gb18030')
basic_info
# 总资产数据，所有公司都有

,Symbol,ShortName,EndDate,AccountsReceivable,TotalCurrentAssets,Inventory,FixedAssets,TotalAssets,TotalCurrentliabilities,TotalLiabilities,...,IG,lev,LEV,OPM,rg,RG,sg,SG,sgee,SGEE
0,1,深发展A,2000,-3.960032e+08,NaN,NaN,1.587091e+09,6.722750e+10,NaN,6.248862e+10,...,NaN,0.929510,NaN,NaN,-3.960032e+08,NaN,0.000000e+00,NaN,NaN,NaN
1,1,深发展A,2001,1.120290e+07,NaN,NaN,1.763257e+09,1.201270e+11,NaN,1.164993e+11,...,NaN,0.969801,1.043347,NaN,1.120290e+07,-0.028290,0.000000e+00,NaN,NaN,NaN
2,1,深发展A,2002,-7.627110e+08,NaN,NaN,2.380640e+09,1.661664e+11,NaN,1.623984e+11,...,NaN,0.977324,1.007757,NaN,-7.627110e+08,-68.081553,0.000000e+00,NaN,NaN,NaN
3,1,深发展A,2003,-8.331532e+08,NaN,NaN,2.388062e+09,1.928510e+11,NaN,1.888859e+11,...,NaN,0.979440,1.002165,NaN,-8.331532e+08,1.092358,0.000000e+00,NaN,NaN,NaN
4,1,深发展A,2004,-8.742457e+08,NaN,NaN,3.243569e+09,2.042864e+11,NaN,1.996018e+11,...,NaN,0.977068,0.997579,NaN,-8.742457e+08,1.049322,0.000000e+00,NaN,NaN,NaN
5,1,深发展A,2005,-7.636040e+08,NaN,NaN,2.417723e+09,2.292164e+11,NaN,2.241734e+11,...,NaN,0.977999,1.000952,NaN,-7.636040e+08,0.873443,0.000000e+00,NaN,NaN,NaN
6,1,深发展A,2006,NaN,NaN,NaN,1.994379e+09,2.605763e+11,NaN,2.541018e+11,...,NaN,0.975153,0.997091,NaN,0.000000e+00,0.000000,0.000000e+00,NaN,NaN,NaN
7,1,深发展A,2007,NaN,NaN,NaN,1.554278e+09,3.525394e+11,NaN,3.395333e+11,...,NaN,0.963107,0.987647,NaN,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN
8,1,深发展A,2008,1.510959e+10,NaN,NaN,1.674924e+09,4.744402e+11,NaN,4.580394e+11,...,NaN,0.965431,1.002413,NaN,1.510959e+10,NaN,0.000000e+00,NaN,NaN,NaN
9,1,深发展A,2009,3.520926e+10,NaN,NaN,1.714461e+09,5.878110e+11,NaN,5.673414e+11,...,NaN,0.965177,0.999736,NaN,3.520926e+10,2.330259,0.000000e+00,NaN,NaN,NaN


In [243]:
#从 basic_info 中获得上市公司的名字，以此筛选是否有ST PT
# industry = pd.read_excel('会计信息质量-财务指标/AIQ_LCFinIndexY.xlsx',encoding="gbk")
Stkcd = basic_info['Symbol']
Stknme = basic_info['ShortName']

# 非ST非PT的上市公司列表,里面存放name-year二元组
ST_PT_except = []
for i in range(basic_info.shape[0]):
    year = basic_info.loc[i,'EndDate']
    if "ST" not in Stknme[i] and "PT" not in Stknme[i]:
        ST_PT_except.append( (Stkcd[i],year) )
print(len(Stkcd),len(ST_PT_except))

52574 50072


In [437]:
ST_PT_except

[(1, 2000),
 (1, 2001),
 (1, 2002),
 (1, 2003),
 (1, 2004),
 (1, 2005),
 (1, 2006),
 (1, 2007),
 (1, 2008),
 (1, 2009),
 (1, 2010),
 (1, 2011),
 (1, 2012),
 (1, 2013),
 (1, 2014),
 (1, 2015),
 (1, 2016),
 (1, 2017),
 (1, 2018),
 (1, 2019),
 (1, 2020),
 (1, 2021),
 (2, 2000),
 (2, 2001),
 (2, 2002),
 (2, 2003),
 (2, 2004),
 (2, 2005),
 (2, 2006),
 (2, 2007),
 (2, 2008),
 (2, 2009),
 (2, 2010),
 (2, 2011),
 (2, 2012),
 (2, 2013),
 (2, 2014),
 (2, 2015),
 (2, 2016),
 (2, 2017),
 (2, 2018),
 (2, 2019),
 (2, 2020),
 (2, 2021),
 (4, 2001),
 (4, 2002),
 (4, 2003),
 (4, 2004),
 (4, 2005),
 (4, 2011),
 (4, 2012),
 (4, 2013),
 (4, 2014),
 (4, 2015),
 (4, 2016),
 (4, 2017),
 (4, 2018),
 (4, 2019),
 (4, 2020),
 (4, 2021),
 (5, 2000),
 (5, 2001),
 (5, 2002),
 (5, 2008),
 (5, 2009),
 (5, 2010),
 (5, 2011),
 (5, 2012),
 (5, 2013),
 (5, 2014),
 (5, 2015),
 (5, 2016),
 (5, 2017),
 (5, 2018),
 (5, 2019),
 (5, 2020),
 (6, 2000),
 (6, 2001),
 (6, 2002),
 (6, 2003),
 (6, 2004),
 (6, 2005),
 (6, 2006),
 (6,

### basic_info里有总资产，所以不用再读取FS_Combas.csv

In [244]:
#读取FS_Combas.csv，获得企业的总资产和总负债
# FS_combas = pd.read_csv('资产合计_资产负债表/FS_Combas.csv',encoding="utf-8")
# FS_combas.head
# FS_combas 用于查询公司的总资产和行业
FS_combas = basic_info.loc[:,['Symbol','ShortName','EndDate','TotalAssets','Industry']].set_index(['Symbol','EndDate'])
FS_combas

ShortName   TotalAssets Industry
Symbol EndDate                                 
1      2000         深发展A  6.722750e+10       金融
       2001         深发展A  1.201270e+11       金融
       2002         深发展A  1.661664e+11       金融
       2003         深发展A  1.928510e+11       金融
       2004         深发展A  2.042864e+11       金融
       2005         深发展A  2.292164e+11       金融
       2006         深发展A  2.605763e+11       金融
       2007         深发展A  3.525394e+11       金融
       2008         深发展A  4.744402e+11       金融
       2009         深发展A  5.878110e+11       金融
       2010         深发展A  7.272071e+11       金融
       2011         深发展A  1.258177e+12       金融
       2012         平安银行  1.606537e+12       金融
       2013         平安银行  1.891741e+12       金融
       2014         平安银行  2.186459e+12       金融
       2015         平安银行  2.507149e+12       金融
       2016         平安银行  2.953434e+12       金融
       2017         平安银行  3.248474e+12       金融
       2018         平安银行  3.418592e+12       金融
       2019         平安银行  3.939070e+12       金融
       2020         平安银行  4.468514e+12       金融
       2021         平安银行  4.921380e+12       金融
2      2000         深万科A  5.622247e+09      房地产
       2001         深万科A  6.482912e+09      房地产
       2002          万科A  8.215822e+09      房地产
       2003          万科A  1.056104e+10      房地产
       2004          万科A  1.553442e+10      房地产
       2005        G 万科A  2.199239e+10      房地产
       2006          万科A  4.850792e+10      房地产
       2007          万科A  1.000945e+11      房地产
...                  ...           ...      ...
900956 2012         东贝B股  3.807986e+09       工业
       2013         东贝B股  3.851675e+09       工业
       2014         东贝B股  4.147086e+09       工业
       2015         东贝B股  4.441134e+09       工业
       2016         东贝B股  4.351075e+09       工业
       2017         东贝B股  4.701589e+09       工业
       2018         东贝B股  4.861189e+09       工业
       2019         东贝B股  5.056674e+09       工业
900957 2000         凌云B股  1.460754e+09       综合
       2001         凌云B股  1.436848e+09       综合
       2002         凌云B股  2.138712e+09       综合
       2003         凌云B股  2.297617e+09       综合
       2004         凌云B股  2.887705e+09       综合
       2005         凌云B股  1.416070e+09       综合
       2006         凌云B股  7.989506e+08       综合
       2007         凌云B股  6.983201e+08       综合
       2008         凌云B股  7.021468e+08       综合
       2009         凌云B股  7.041906e+08       综合
       2010         凌云B股  4.926711e+08       综合
       2011         凌云B股  4.308414e+08       综合
       2012         凌云B股  3.914345e+08      房地产
       2013         凌云B股  3.954345e+08      房地产
       2014         凌云B股  8.872896e+08      房地产
       2015         凌云B股  1.015625e+09      房地产
       2016         凌云B股  1.165033e+09      房地产
       2017         凌云B股  1.019428e+09      房地产
       2018         凌云B股  1.007002e+09      房地产
       2019         凌云B股  1.017715e+09      房地产
       2020         凌云B股  1.008562e+09      房地产
       2021         凌云B股  1.023537e+09      房地产

[52574 rows x 3 columns]

In [97]:
from statistics import mean
# 整合每个公司每个欺诈年份的欺诈类型
'''
def match_asset_by_date(stkID, year):
    # 默认已经提前读进FS_Combas.csv文件并保存在FS_combas中
    all_time_index = []
    for i in range(FS_combas.shape[0]):
        if FS_combas.loc[i,'Stkcd'] == stkID:
            all_time_index.append(i)
        elif FS_combas.loc[i,'Stkcd'] > stkID:
            break
    #默认的总资产是全时间段最后一个，防止找不到对应年份的总资产值
    #asset_amount = FS_combas.loc[all_time_index[-1],'A001000000']
    assets_by_year = []
    for i in all_time_index:
        # 找到股票代码相匹配并且报表类型是A：“A＝合并报表”、“B＝母公司报表”
        if FS_combas.loc[i,'Stkcd'] == stkID and FS_combas.loc[i,'Typrep'] == "A":
            if int(FS_combas.loc[i,'Accper'].split('-')[0]) == year:
                #找到同一年的，并且把所有该年份的asset取平均
                assets_by_year.append(FS_combas.loc[i,'A001000000'])
    try: 
        asset_amount = mean(assets_by_year)
    except:
        asset_amount = 0
    return asset_amount'''

def judge_ST_PT(stkID, year):
    years = [int(year)-i for i in range(6)]
    for y in years:
        if (stkID,y) not in ST_PT_except:
            return False
    return True

judge_ST_PT(7,2011)

False

In [245]:
firm_year_type = {}
firm_year_asset = {}

'''统计各筛选阶段中的淘汰公司数量'''
n_fraud_without_time, n_corp_without_ind, n_fincorp, n_corp_fraud = [], [], [], []


for i in range(weigui.shape[0]):
    stkID = weigui.loc[i,'Symbol']
    if weigui.loc[i,'ViolationYear'] is not np.nan:
        n_fraud_without_time.append(stkID)
        
        vio_years = json.loads(weigui.loc[i,'ViolationYear'])
        '''# 排除最晚1999年以前的造假
        if vio_years[-1] > 1999:
            # 保留2000年以来的造假
            vio_years = [j for j in vio_years if j > 1999]'''
        for year in vio_years:
            if (stkID, year) in FS_combas.index:
                n_corp_without_ind.append(stkID)
                # 获取行业
                ind = FS_combas.loc[(stkID,year),'Industry']
                #  排除行业类型为金融的，ST 或者 PT暂时先不判断
                # if judge_ST_PT(stkID,year) is True
                if '金融' not in ind:
                    n_fincorp.append(stkID)
                    # 获取违规类型，保存为list
                    weiguiType = str(weigui.loc[i,'ViolationType']).split(',') #str()可以将nan转化为'nan',因此就不会报错
                    # print(stkID,year,weiguiType)
                    if "虚假记载" in weiguiType or "虚列资产" in weiguiType or "虚构利润" in weiguiType or "披露不实" in weiguiType or "重大遗漏" in weiguiType or '虚假记载(误导性陈述)' in weiguiType or '披露不实(其它)' in weiguiType :
                        n_corp_fraud.append(stkID)
                        if stkID not in firm_year_type:
                            firm_year_type[stkID] = {}
                        if year in firm_year_type[stkID]:
                            # 将当前年份的违规类型合并到firm_year_type[stkID][year]中,
                            firm_year_type[stkID][year] = list(set(firm_year_type[stkID][year] + weiguiType))
                        else:
                            firm_year_type[stkID][year] = weiguiType
                        firm_year_asset[(stkID, year)]= FS_combas.loc[(stkID, year),'TotalAssets']
                        
print('共有%d次强制执行有表明欺诈日期,剩下%d家公司的欺诈没有日期' % (len(set(n_fraud_without_time)), len(set(weigui.Symbol)-set(n_fraud_without_time))))
print('共有%d家公司有行业数据,剩下%d家公司没有行业数据' % (len(set(n_corp_without_ind)), len(set(n_fraud_without_time)-set(n_corp_without_ind))))
print('共有%d家违规非金融公司,剩下%d家公司为金融公司' % (len(set(n_fincorp)), len(set(n_corp_without_ind)-set(n_fincorp))))
print('共有%d家公司违规类是财务报表欺诈,剩下%d家公司没有发生财务报表欺诈' % (len(set(n_corp_fraud)), len(set(n_fincorp)-set(n_corp_fraud))))

共有2765次强制执行有表明欺诈日期,剩下60家公司的欺诈没有日期
共有2749家公司有行业数据,剩下16家公司没有行业数据
共有2708家违规非金融公司,剩下41家公司为金融公司
共有1718家公司违规类是财务报表欺诈,剩下990家公司没有发生财务报表欺诈


In [246]:
# 筛选合格的上市公司,生成frau_time_industry_money_2021.csv
# def generate_fraud_info():
'''with open('frau_time_industry_money_til2021.csv', 'a+', encoding='gbk', newline='') as file:
    for stkID in firm_year_type:
        ind = ind_dict[stkID]
        for year in firm_year_type[stkID]:
            weigui_type = json.dumps(firm_year_type[stkID][year])
            asset = firm_year_asset[(stkID, year)]
            file.write(str(stkID).zfill(6) + ',' + str(year) + ',' + ind + ',' + weigui_type + ',' + str(asset) +'\r\n')'''
frau_time_industry_money = pd.DataFrame(columns=['Stkcd','inc_name','fraud_year','industry','type','asset'])
for stkID in firm_year_type:
    for year in firm_year_type[stkID]:
        ind = FS_combas.loc[(stkID,year),'Industry']
        inc_name = FS_combas.loc[(stkID,year),'ShortName']
        # ensure_ascii=False 防止出现中文乱码
        weigui_type = json.dumps(firm_year_type[stkID][year],ensure_ascii=False)
        asset = firm_year_asset[(stkID, year)]
        temp = pd.DataFrame({
            'Stkcd':str(stkID).zfill(6),
            'inc_name':inc_name,
            'fraud_year':year,
            'industry':ind,
            'type':weigui_type,
            'asset':asset
        },index=[0])
        frau_time_industry_money = frau_time_industry_money.append(temp,ignore_index=True)

In [247]:
# frau_time_industry_money
frau_time_industry_money.to_csv('frau_time_industry_money_til2021.csv',index=None,encoding='gb18030')

In [249]:
fraud_info = pd.read_csv('frau_time_industry_money_til2021.csv',encoding='gb18030')
len(set(fraud_info.loc[:,'Stkcd']))
#fraud_info.head

1718

In [153]:
'''#生成用于未违规公司采样的数据 -> all_time_industry_money_til2021.csv
weiguiID = set(fraud_info.loc[:,'Stkcd'])
with open('all_time_industry_money_til2021.csv', 'a+', encoding='gbk', newline='') as file:
    file.write('Stkcd,inc_name,disposalDate,industry,asset\r\n')
    for i in range(basic_info.shape[0]):
        stkID = basic_info.loc[i,'Symbol']
        year = basic_info.loc[i,'EndDate']
        inc_name = FS_combas.loc[(stkID,year),'ShortName']
        asset = FS_combas.loc[(stkID,year),'TotalAssets']
        # 没有任何类型的违规记录且不属于ST PT公司
        if stkID not in weiguiID and (stkID,year) in ST_PT_except:
            ind = FS_combas.loc[(stkID,year),'Industry']
            # 行业排除金融
            if ind != "金融":
                file.write(str(stkID).zfill(6) + ',' + inc_name + ',' + str(year) + ',' + ind + ',' + str(asset) +'\r\n')'''

In [156]:
'''isin = pd.read_csv('ISIN公司代码/STK_LISTEDCOINFOANL.csv',sep=',')
isin_dict = {}
for i in range(isin.shape[0]):
    isin_dict[isin.loc[i,'Symbol']] = isin.loc[i,'ISIN']
print(len(isin_dict))

#生成 compustat 的目标公司ISIN代码,用于输入到 compustat 中查看,并在csv中添加isin码
df1 = pd.read_csv('frau_time_industry_money_til2021.csv',encoding='gbk')
df2 = pd.read_csv('all_time_industry_money_til2021.csv',encoding='gbk')
print(len(set(df1.loc[:,'Stkcd'])),len(set(df2.loc[:,'Stkcd'])))'''

4422
1718 3146


### 下面的操作是为了链接到compustat上收集财务指标，但由于财务指标在国泰安里已有，所以不必须

In [157]:
# 加上ISIN 码，不必须
for i in range(df1.shape[0]):
    if df1.loc[i,'Stkcd'] in isin_dict:
        df1.loc[i,'isin'] = isin_dict[df1.loc[i,'Stkcd']]
for i in range(df2.shape[0]):
    if df2.loc[i,'Stkcd'] in isin_dict:
        df2.loc[i,'isin'] = isin_dict[df2.loc[i,'Stkcd']]

#df1.to_csv('frau_time_industry_money_til2021.csv',index=None,encoding='gbk')
#df2.to_csv('all_time_industry_money_til2021.csv',index=None,encoding='gbk')

In [187]:
query_by_isin = pd.DataFrame(columns=['ISIN'])
symbol_list = list(list(set(df1.iloc[:,0])) + list(set(df2.iloc[:,0])))
isin_list = []
for x in symbol_list:
    try:
        isin_list.append(isin_dict[x])
    except:
        print(str(x).zfill(6) + '没有ISIN码')

query_by_isin['ISIN'] = isin_list
query_by_isin.to_csv('compustat_query_by_isin.csv' , header=None)

200002没有ISIN码
200012没有ISIN码
200016没有ISIN码
200018没有ISIN码
200019没有ISIN码
200022没有ISIN码
200025没有ISIN码
200026没有ISIN码
200029没有ISIN码
200037没有ISIN码
200039没有ISIN码
200045没有ISIN码
200055没有ISIN码
200056没有ISIN码
200058没有ISIN码
200413没有ISIN码
200418没有ISIN码
200429没有ISIN码
200488没有ISIN码
200505没有ISIN码
200513没有ISIN码
200521没有ISIN码
200530没有ISIN码
200539没有ISIN码
200541没有ISIN码
200550没有ISIN码
200553没有ISIN码
200570没有ISIN码
200581没有ISIN码
200596没有ISIN码
200625没有ISIN码
200726没有ISIN码
200761没有ISIN码
200869没有ISIN码
201872没有ISIN码
900901没有ISIN码
900902没有ISIN码
900903没有ISIN码
900904没有ISIN码
900905没有ISIN码
900906没有ISIN码
900907没有ISIN码
900908没有ISIN码
900909没有ISIN码
900910没有ISIN码
900911没有ISIN码
900912没有ISIN码
900913没有ISIN码
900914没有ISIN码
900915没有ISIN码
900916没有ISIN码
900917没有ISIN码
900920没有ISIN码
900921没有ISIN码
900922没有ISIN码
900923没有ISIN码
900924没有ISIN码
900925没有ISIN码
900926没有ISIN码
900927没有ISIN码
900928没有ISIN码
900932没有ISIN码
900933没有ISIN码
900934没有ISIN码
900937没有ISIN码
900938没有ISIN码
900940没有ISIN码
900941没有ISIN码
900942没有ISIN码
900943没有ISIN码
900947没有ISIN码
900952

### 以上的操作是为了链接到compustat上收集财务指标，但由于财务指标在国泰安里已有，所以以上操作不必须

In [193]:
# 这种方法生成的code_for_CSMAR是错的
'''query_by_Stkcd = pd.DataFrame(columns=['code'])
# 把有 isin 码的 symbol 筛出来，用于国泰安的查询
query_by_Stkcd['code'] = [str(x).zfill(6) for x in symbol_list if x in isin_dict]
query_by_Stkcd.sort_values(by="code" , inplace=True, ascending=True) 
query_by_Stkcd.to_excel('code_for_CSMAR.xlsx',index=None)'''

## 老的基于compustat的指标用以下代码计算，基于国泰安的12个财务指标是在excel里直接算出来的，这里用国泰安的

In [4]:
'''raw_finData = pd.read_csv('base_index_from_compustat.csv',encoding='gb18030')
# 1.计算AQI,AQI是除固定资产之外的非流动资产占总资产的比例
# AQI is the ratio of non-current assets other than property, plant, and equipment, to total assets, for time period t relative to time period t-1. 
# An AQI greater than 1 indicates that the firm has potentially increased its involvement in cost deferral, a possible indicator of asset overstatement fraud
for i in range(raw_finData.shape[0]-1):
    isin = raw_finData.loc[i,'isin']
    if isin != raw_finData.loc[i+1,'isin']:
        continue
    at_t_1 = raw_finData.loc[i, 'at']
    at_t = raw_finData.loc[i+1, 'at']
    act_t_1 = raw_finData.loc[i,'act']
    act_t = raw_finData.loc[i+1,'act']
    ppent_t_1 = raw_finData.loc[i,'ppent']
    ppent_t = raw_finData.loc[i+1,'ppent']
    if at_t_1==0 or at_t==0 or str(at_t_1)== 'nan' or str(at_t)== 'nan' or str(act_t_1)== 'nan' or str(act_t)== 'nan' or str(ppent_t_1)== 'nan' or str(ppent_t)== 'nan':
        AQI = 1
    else:
        AQI = (at_t-act_t-ppent_t)/(at_t_1-act_t_1-ppent_t_1) * at_t_1 / at_t
    raw_finData.loc[i+1,'AQI'] = AQI
# 2.计算资产周转率 Asset Turnover(AT)
# AT is the ratio of net sales to total assets. When revenue fraud is being committed, 
# net sales are often increased artificially and rapidly, resulting in a large AT value
for i in range(raw_finData.shape[0]):
    sale_t = raw_finData.loc[i,'sale']
    at_t = raw_finData.loc[i, 'at']

    if sale_t == 0 or at_t ==0 or str(at_t)== 'nan' or str(sale_t)== 'nan':
        AT = 1
    else:
        AT = sale_t / at_t
    raw_finData.loc[i,'AssetTurnover'] = AT

# 3.计算现金流盈余差 Cash Flow Earnings Difference (CFED)
# FED assesses the impact of accruals on financial statements (Beneish 1999a; Dechow et al. 2011). 
# This ratio is often positive when revenue fraud is occurring or when employees are engaging in cash theft.
for i in range(raw_finData.shape[0]):
    oancf_t = raw_finData.loc[i,'oancf']
    nicon_t = raw_finData.loc[i,'nicon']
    if str(oancf_t)=='nan' or str(nicon_t)=='nan':
        continue
        #CFED = None
    else:
        CFED = oancf_t-nicon_t
    raw_finData.loc[i,'CFED'] = CFED

# 4.计算应收账款销售天数 Days Sales in Receivables (DSRI)
# DSIR is the ratio of day sales in receivables in period t to the corresponding measure in period t-1. 
# Firms engaging in revenue fraud often add fictitious receivables, causing DSIR to increase.
for i in range(raw_finData.shape[0]-1):
    isin = raw_finData.loc[i,'isin']
    if isin != raw_finData.loc[i+1,'isin']:
        continue
    sale_t = raw_finData.loc[i+1,'sale']
    sale_t_1 = raw_finData.loc[i,'sale']
    rect_t = raw_finData.loc[i+1,'rect']
    rect_t_1 = raw_finData.loc[i,'rect']
    if sale_t==0 or sale_t_1==0 or rect_t==0 or rect_t_1==0 or str(sale_t)=='nan' or str(sale_t_1)=='nan' or str(rect_t)=='nan' or str(rect_t_1)=='nan':
        DSIR = 1
    else:
        DSIR = rect_t/rect_t_1 * sale_t_1/sale_t
    raw_finData.loc[i+1,'DSIR'] = DSIR

# 5.折旧指数 Depreciation Index (DEPI)
# DEPI is the ratio of the rate of depreciation in period t-1 as compared to period t. 
# Fictitious assets accelerate the depreciation rate, resulting in smaller values for DEPI
for i in range(raw_finData.shape[0]-1):
    isin = raw_finData.loc[i,'isin']
    if isin != raw_finData.loc[i+1,'isin']:
        continue
    dp_t_1 = raw_finData.loc[i,'dp']
    dp_t = raw_finData.loc[i+1,'dp']
    am_t_1 = raw_finData.loc[i,'am']
    am_t = raw_finData.loc[i+1,'am']
    ppent_t_1 = raw_finData.loc[i,'ppent']
    ppent_t = raw_finData.loc[i+1,'ppent']
    if str(dp_t_1) == 'nan' or str(dp_t) == 'nan' or str(am_t_1) == 'nan' or str(am_t) == 'nan' or str(ppent_t_1) == 'nan' or str(ppent_t) == 'nan':
        DEPI = 1
    else:
        # !!! 是用t-1/t
        DEPI = (dp_t_1 - am_t_1)/(dp_t - am_t) * (dp_t - am_t + ppent_t)/(dp_t_1 - am_t_1 + ppent_t_1)
    raw_finData.loc[i+1,'DEPI'] = DEPI

# 6.毛利率指数 Gross Margin Index (GMI)
# GMI is the ratio of the gross margin in period t-1 to the gross margin in period t.
# A GMI greater than 1 suggests that gross margins have deteriorated, a condition rarely encountered when a firm is engaging in revenue fraud.
for i in range(raw_finData.shape[0]-1):
    isin = raw_finData.loc[i,'isin']
    if isin != raw_finData.loc[i+1,'isin']:
        continue
    sale_t_1 = raw_finData.loc[i,'sale']
    sale_t = raw_finData.loc[i+1,'sale']
    cogs_t_1 = raw_finData.loc[i,'cogs']
    cogs_t = raw_finData.loc[i+1,'cogs']
    if str(sale_t_1) == 'nan' or str(sale_t) == 'nan' or str(cogs_t_1) == 'nan' or str(cogs_t) == 'nan' or sale_t_1 == 0 or sale_t==0:
        GMI = 1
    else:
        # !!! 是(t-1)/t
        GMI = (sale_t_1 - cogs_t_1)/(sale_t-cogs_t) * sale_t/sale_t_1
    raw_finData.loc[i+1,'GMI'] = GMI

# 7.库存增长 Inventory Growth(IG)
# IG assesses whether inventory has grown in period t as compared to period t-1. 
# IG is used to detect whether ending inventory is being over- stated to decrease cost of goods sold and increase gross margin.
for i in range(raw_finData.shape[0]-1):
    isin = raw_finData.loc[i,'isin']
    if isin != raw_finData.loc[i+1,'isin']:
        continue
    invt_t = raw_finData.loc[i+1,'invt']
    invt_t_1 = raw_finData.loc[i,'invt']
    if str(invt_t) =='nan' or str(invt_t_1) =='nan' or invt_t==0 or invt_t_1==0:
        IG = 1
    else:
        IG = invt_t/invt_t_1
    raw_finData.loc[i+1,'IG'] = IG

# 8.杠杆 leverage index(LEV)
# LEV is the ratio of total debt to total assets in period t relative to period t-1. 
# LEV is used to detect whether firms are fictitiously including assets on the balance sheet without any corresponding debt.
for i in range(raw_finData.shape[0]-1):
    isin = raw_finData.loc[i,'isin']
    if isin != raw_finData.loc[i+1,'isin']:
        continue
    dltt_t_1 = raw_finData.loc[i,'dltt']
    dltt_t = raw_finData.loc[i+1,'dltt']
    lct_t_1 = raw_finData.loc[i,'lct']
    lct_t = raw_finData.loc[i+1,'lct']
    at_t_1 = raw_finData.loc[i,'at']
    at_t = raw_finData.loc[i+1,'at']
    if str(dltt_t_1)=='nan' or str(dltt_t)=='nan' or str(lct_t_1)=='nan' or str(lct_t)=='nan' or str(at_t_1)=='nan' or str(at_t)=='nan' or at_t_1==0 or at_t==0:
        LEV = 1
    else:
        LEV = (dltt_t+lct_t)/at_t * at_t_1/(dltt_t_1+lct_t_1)
    raw_finData.loc[i+1,'LEV'] = LEV

# 9.经营业绩保证金 Operating Performance Margin (OPM)
# OPM is calculated by dividing net income by net sales. 
# When fraudulent firms add fictitious sales revenues, they often fall to the bottom line without additional costs, thus inflating the value of OPM.
for i in range(raw_finData.shape[0]):
    nicon_t = raw_finData.loc[i,'nicon']
    sale_t = raw_finData.loc[i,'sale']
    if str(sale_t)=='nan' or str(nicon_t)=='nan' or sale_t==0 or nicon_t==0:
        OPM = 1
    else:
        OPM = nicon_t/sale_t
    raw_finData.loc[i,'OPM'] = OPM

# 10.应收帐款增长 Receivables Growth (RG)
# RG is the amount of receivables in period t divided by the amount in period t-1.
# Firms engaging in revenue fraud often add fictitious revenues and receivables, thereby increasing RG.
for i in range(raw_finData.shape[0]-1):
    isin = raw_finData.loc[i,'isin']
    if isin != raw_finData.loc[i+1,'isin']:
        continue
    rect_t = raw_finData.loc[i+1,'rect']
    rect_t_1 = raw_finData.loc[i,'rect']
    if str(rect_t)=='nan' or str(rect_t_1)=='nan' or rect_t==0 or rect_t_1==0:
        RG = 1
    else:
        RG = rect_t/rect_t_1
    raw_finData.loc[i+1,'RG'] = RG

# 11.销售增长 Sales Growth Index (SGI)
# SG is equal to net sales in period t divided by net sales in period t-1. 
# In the presence of revenue fraud, the value of SG generally increases.
for i in range(raw_finData.shape[0]-1):
    isin = raw_finData.loc[i,'isin']
    if isin != raw_finData.loc[i+1,'isin']:
        continue
    sale_t = raw_finData.loc[i+1,'sale']
    sale_t_1 = raw_finData.loc[i,'sale']
    if str(sale_t)=='nan' or str(sale_t_1)=='nan' or sale_t==0 or sale_t_1==0:
        SG = 1
    else:
        SG = sale_t/sale_t_1
    raw_finData.loc[i+1,'SG'] = SG

# 12.SGE费用: SGE Expense (SGEE)
# SGEE is calculated by dividing the ratio of selling and general administrative expenses to net sales in period t by the same ratio in period t-1. 
# When firms are engaging in revenue fraud, SGE expenses represent a smaller percentage of the artificially inflated net sales, there- by causing SGEE to decrease.
for i in range(raw_finData.shape[0]-1):
    isin = raw_finData.loc[i,'isin']
    if isin != raw_finData.loc[i+1,'isin']:
        continue
    xsga_t = raw_finData.loc[i+1,'xsga']
    xsga_t_1 = raw_finData.loc[i,'xsga']
    sale_t = raw_finData.loc[i+1,'sale']
    sale_t_1 = raw_finData.loc[i,'sale']
    if xsga_t==0 or xsga_t_1==0 or sale_t==0 or sale_t_1==0 or str(xsga_t)=='nan' or str(xsga_t_1)=='nan' or str(sale_t)=='nan' or str(sale_t_1)=='nan':
        SGEE = 1
    else:
        SGEE = xsga_t/xsga_t_1 * sale_t_1/sale_t
    raw_finData.loc[i+1,'SGEE'] = SGEE

#raw_finData.to_csv('finRatio.csv')'''

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars


In [250]:
# 读取基于国泰安的12个财务指标，然后把每个公司已记录的第一年指标全设为 Nan
finRatio = pd.read_excel('会计信息质量-财务指标/AIQ_LCFinIndexY.xlsx',encoding='gb18030')
# 把第一年的指标归零
pre_firm = ''
for i in range(finRatio.shape[0]):
    firm = finRatio.loc[i,'Symbol']
    if firm != pre_firm:
        # 把所有的 指标设为 nan
        finRatio.iloc[i,25:] = np.nan
    pre_firm = firm
finRatio.to_excel('会计信息质量-财务指标/AIQ_LCFinIndexY_used.xlsx',index=None,encoding='gb18030')

In [262]:
# 读取12个财务指标，将all_time_industry_money_til2021 和 finRatio 合并
finRatio = pd.read_excel('会计信息质量-财务指标/AIQ_LCFinIndexY_used.xlsx',encoding='gb18030')

# 读取 all_time_industry_money_til2021.csv
fraud_info = pd.read_csv('frau_time_industry_money_til2021.csv',encoding='gbk')
finRatio = finRatio.set_index(['Symbol','EndDate'])


In [252]:
fraud_firm_series = pd.DataFrame(columns=['Stkcd','year','is_fraud','AQI','AT','CFED','DSIR','DEPI','GMI','IG','LEV','OPM','RG','SG','SGEE'])
fraud_firm_series = fraud_firm_series.set_index(['Stkcd','year'])

# 生成欺诈年份以前的非欺诈年份的序列
def generate_time_series(list1):
    # 从欺诈年份往前数至多5年的数据
    list2 = [(list1[0],True)]
    for i in range(len(list1)-1):
        interval = (list1[i] - list1[i+1])
        fill_size = 0
        if interval == 1:
            list2.append((list1[i+1],True))
            continue
        else:
            if interval >= 5:
                fill_size = 5
            else:
                # 0 < interval < 5
                fill_size = interval
            for j in range(fill_size-1):
                list2.append((list1[i]- j-1, False))
            return list2
    
    for i in range(5):
        list2.append((list1[-1]-i-1,False))
    return list2

def add_row(symbol, year_fraud_list):
    for year, is_fraud in year_fraud_list:
        if (symbol,year) in finRatio.index:
            data = finRatio.loc[(symbol,year),['AQI','AT','CFED','DSIR','DEPI','GMI','IG','LEV','OPM','RG','SG','SGEE']]
            fraud_firm_series.loc[(symbol,year),'is_fraud'] = is_fraud
            fraud_firm_series.loc[(symbol,year),['AQI','AT','CFED','DSIR','DEPI','GMI','IG','LEV','OPM','RG','SG','SGEE']] = data
            # [d for d in data if d is not np.nan and d != '#DIV/0!']
    return 0

years = []
pre_sym = fraud_info.loc[0,'Stkcd']

for i in range(fraud_info.shape[0]):
    symbol = fraud_info.loc[i,'Stkcd']
    year = fraud_info.loc[i,'fraud_year']
    
    if symbol == pre_sym:
        years.append(year)
    else:
        year_fraud = generate_time_series(sorted(years,reverse=True))
        add_row(pre_sym, year_fraud)
        # 更新
        pre_sym = symbol
        years = [year]
    
    if i+1 == fraud_info.shape[0]: # 最后一行
        year_fraud = generate_time_series(sorted(years,reverse=True))
        add_row(pre_sym, year_fraud)

fraud_firm_series.to_csv('fraud_series.csv', encoding='utf-8')

In [256]:
fraud_firm_series.loc[20,:]

Stkcd                 5
year               2021
is_fraud           True
AQI             1.03337
AT             0.155003
CFED        1.39633e+08
DSIR           0.532014
DEPI             1.0155
GMI            0.936686
IG              1.08107
LEV            0.878964
OPM            0.363619
RG             0.511496
SG             0.961434
SGEE            0.85589
Name: 20, dtype: object

In [323]:
# fraud_series.csv -> fraud_series_net.csv：把缺少财务指标的公司去了，
from pandas import isnull


fraud_firm_series = pd.read_csv('fraud_series.csv', encoding='utf-8')
print('初始:共有%d家公司' % len(set(fraud_firm_series.Stkcd)))

n_no_nonfraud_year, n_lack_findata = set(), set()

drop_index = []
# 先把 含有 Nan 的行删掉。
for i in range(fraud_firm_series.shape[0]):
    if fraud_firm_series.loc[i,:].isnull().tolist().count(True)>=11:
        # 一整行都是null的firm-year，说明是该公司的第一年
        drop_index.append(i)
    elif int(fraud_firm_series.loc[i,'year']) < 2003:
        # 把 2003年 前的数据删掉，因为2003年以前没有关系数据
        drop_index.append(i)

fraud_firm_series = fraud_firm_series.drop(drop_index,axis=0)
# drop = True, 是否将原索引作为新的一列补进表格中
fraud_firm_series = fraud_firm_series.reset_index(drop = True)

print('删完null的行以及2003年以前的行以后,剩%d家公司，被删的公司只有一行数据且都为null' % len(set(fraud_firm_series.Stkcd)))

#########################################################################################################################################################

# 再drop is_fraud 没有 false，全是 true 的公司, 然后drop含有 0 和 #DIV/0! 的公司 (缺少财务指标的公司)
'''统计每次筛选筛掉的公司数量,写进论文里'''

series_by_firm = fraud_firm_series.set_index(['Stkcd'])
drop_index= set()

for firm in set(series_by_firm.index):
    df = series_by_firm.loc[firm, :]
    if df.shape == (14,):
        drop_index.add(firm)
        n_no_nonfraud_year.add(firm)

    elif df.iloc[-1,1]==True:
        # 去掉没有非欺诈时间序列的公司
        drop_index.add(firm)
        n_no_nonfraud_year.add(firm)

    # elif df[(df.DSIR=='#DIV/0!')|(df.DEPI=='#DIV/0!')|(df.GMI=='#DIV/0!')|(df.IG=='#DIV/0!')|(df.RG=='#DIV/0!')].shape[0]>0:
    elif df.isnull().sum().sum()>0:
        # 去掉财务指标缺失值的公司
        # 在AIQ_LCFinIndexY_used.xlsx（excel）文件中，不存在“#DIV/0!”的值，取而代之的全部是空值。只有在csv文件中的才是“#DIV/0!”，所以这个条件判断没有用
        
        drop_index.add(firm)
        n_lack_findata.add(firm)

print('共有%d家没有非欺诈年份数据的公司,剩下%d家公司有非欺诈年份数据' % (len(n_no_nonfraud_year), len(set(series_by_firm.index)-n_no_nonfraud_year)))
print('共有%d家公司缺失部分财务指标,剩下%d家公司财务指标完好' % (len(n_lack_findata),len(set(series_by_firm.index)- n_no_nonfraud_year - n_lack_findata)))

series_by_firm = series_by_firm.drop(drop_index,axis=0)


初始:共有1718家公司
删完null的行以及2003年以前的行以后,剩1652家公司，被删的公司只有一行数据且都为null
共有194家没有非欺诈年份数据的公司,剩下1458家公司有非欺诈年份数据
共有114家公司缺失部分财务指标,剩下1344家公司财务指标完好


In [324]:
# 将行业、规模等数据补到series_by_firm上
# 总共有多少家公司
finRatio = pd.read_excel('会计信息质量-财务指标/AIQ_LCFinIndexY_used.xlsx',encoding='gb18030').set_index(['Symbol','EndDate'])

series_by_firm = series_by_firm.reset_index(drop = False).set_index(['Stkcd','year'])
for firm,year in series_by_firm.index:
    # print(firm,year)
    series_by_firm.loc[(firm,year),'industry'] = finRatio.loc[(firm,year),'Industry']
    series_by_firm.loc[(firm,year),'asset'] = finRatio.loc[(firm,year),'TotalAssets']
series_by_firm.to_csv('fraud_series_net.csv',encoding='gb18030')

In [325]:
'''# 读取管理层讨论与分析的数据
mda = pd.read_excel('MD&A/BDT_MDAEmotAnal_features_raw.xlsx',encoding='gbk')
mda.drop([0,1],axis=0,inplace=True)
merged_mda = pd.DataFrame(columns=['Symbol','EndDate','TextualSimilarity','PositiveVocabularyNum','NegativeVocabularyNum','TotalWordsNum','SentencesNum','WordsNum','EmotionTone1','EmotionTone2'])
merged_mda.set_index(['Symbol','EndDate'],inplace=True)

# 处理年份信息
mda.EndDate = [d.split('-')[0] for d in mda.EndDate.to_list()]
mda = mda.set_index(['Symbol','EndDate'])
for firm, year in mda.index:
    merged_mda.loc[(firm,year),:] = mda.loc[(firm, year),['TextualSimilarity','PositiveVocabularyNum','NegativeVocabularyNum','TotalWordsNum','SentencesNum','WordsNum','EmotionTone1','EmotionTone2']].mean().to_list()
    
merged_mda.to_csv('MD&A/BDT_MDAEmotAnal_features.csv',encoding='gbk')'''

# 将管理层讨论与分析的数据补到series_by_firm上
series_by_firm = pd.read_csv('fraud_series_net.csv',encoding='gb18030')
new_cols = ['TextualSimilarity','PositiveVocabularyNum','NegativeVocabularyNum','TotalWordsNum','SentencesNum','WordsNum','EmotionTone1','EmotionTone2']
for i in new_cols:
    # 初始化新的列
    series_by_firm[i] = np.nan
series_by_firm.set_index(['Stkcd','year'],inplace=True)
mda = pd.read_csv('MD&A/BDT_MDAEmotAnal_features.csv',encoding='gbk')
mda.set_index(['Symbol','EndDate'],inplace=True)

for firm, year in set(series_by_firm.index):
    tuple = (firm, year)
    if tuple in mda.index:
        series_by_firm.loc[tuple,new_cols] = mda.loc[tuple,new_cols]
series_by_firm.to_csv('fraud_series_net.csv',encoding='gb18030')

In [327]:
# 重新生成 all_time_industry_money_til2021.csv
pos_sampling = pd.DataFrame(columns=['Stkcd','EndDate','TotalAssets','OperatingRevenue','Industry','AQI', 'AT', 'CFED', 'DSIR', 'DEPI', 'GMI', 'IG', 'LEV',
       'OPM', 'RG', 'SG', 'SGEE'])
# 把表头写进csv文件
pos_sampling.to_csv('all_time_industry_money_til2021.csv',encoding='gb18030',index=None)
pos_sampling = pos_sampling.set_index(['Stkcd'])

drop_index_1, drop_index_2 = [], []
basic_info = pd.read_excel('会计信息质量-财务指标/AIQ_LCFinIndexY_used.xlsx',encoding='gb18030')
for i in range(basic_info.shape[0]):
   # 把每家公司第一行为null的行删掉
   # 之前搞错了，把所有列中但凡有null的行都删了
   if basic_info.loc[i,['TotalAssets','Industry','AQI', 'AT', 'CFED', 'DSIR', 'DEPI', 'GMI', 'IG', 'LEV','OPM', 'RG', 'SG', 'SGEE']].isnull().tolist().count(True) >= 12:
      drop_index_1.append(i)
   elif int(basic_info.loc[i,'EndDate']) < 2003:
      drop_index_2.append(i)

print('共有%d行的数据总资产、行业和12个指标全为空' % len(drop_index_1))
print('共有%d行的数据在2003年以前' % len(drop_index_2))

basic_info = basic_info.drop(drop_index_1+drop_index_2, axis=0)
basic_info = basic_info.set_index(['Symbol'])

# 排除违规的公司
# fraud_info = pd.read_csv('frau_time_industry_money_til2021.csv',encoding='gb18030')
n_fin_corp = set()
n_no_weigui = set()
n_only_one_year = set()
n_missing_data = set()
weiguiID = set(fraud_info.reset_index().Stkcd)
for firm in sorted(set(basic_info.index)):
   df = basic_info.loc[firm,['EndDate','TotalAssets','OperatingRevenue','Industry','AQI', 'AT', 'CFED', 'DSIR', 'DEPI', 'GMI', 'IG', 'LEV','OPM', 'RG', 'SG', 'SGEE']]
   if df.shape==(16,):
      n_only_one_year.add(firm)
      continue
   ind = df.Industry.to_list()
   if '金融' not in ind:
      n_fin_corp.add(firm)
      if firm not in weiguiID:
         n_no_weigui.add(firm)
         # if df[(df.DSIR=='#DIV/0!')|(df.DEPI=='#DIV/0!')|(df.GMI=='#DIV/0!')|(df.IG=='#DIV/0!')|(df.RG=='#DIV/0!')].shape[0]<1:
         if df.isnull().sum().sum()==0:
            # 判断指标缺失的公司
            # pos_sampling = pos_sampling.append(df,ignore_index=False)
            n_missing_data.add(firm)
            df.to_csv('all_time_industry_money_til2021.csv',encoding='gb18030',mode='a',header=None)

print('初始公司有%s家' % len(set(basic_info.index)))
print('共有%d家公司只有一年数据' % len(n_only_one_year))
print('共有%d家公司不是金融公司' % len(n_fin_corp))
print('共有%d家公司没有财务报表欺诈' % len(n_no_weigui))
print('共有%d家公司没有缺失指标' % len(n_missing_data))


共有4968行的数据总资产、行业和12个指标全为空
共有2239行的数据在2003年以前
初始公司有4500家
共有482家公司只有一年数据
共有3899家公司不是金融公司
共有2224家公司没有财务报表欺诈
共有2081家公司没有缺失指标


In [ ]:
'''共有4968行的数据总资产、行业和12个指标全为空
初始公司有4513家
共有481家公司只有一年数据
共有3913家公司不是金融公司
共有2230家公司没有财务报表欺诈
共有2081家公司没有缺失指标'''

In [328]:
# 将管理层讨论与分析的数据补到 all_time_industry_money_til2021 上
mda = pd.read_csv('MD&A/BDT_MDAEmotAnal_features.csv',encoding='gbk')
mda.set_index(['Symbol','EndDate'],inplace=True)
new_cols = ['TextualSimilarity','PositiveVocabularyNum','NegativeVocabularyNum','TotalWordsNum','SentencesNum','WordsNum','EmotionTone1','EmotionTone2']

pos_sampling = pd.read_csv('all_time_industry_money_til2021.csv',encoding='gb18030')
for i in new_cols:
    pos_sampling[i] = np.nan
pos_sampling.set_index(['Stkcd','EndDate'],inplace = True)
for firm, year in set(pos_sampling.index):
    tuple = (firm, year)
    if tuple in mda.index:
        pos_sampling.loc[tuple,new_cols] = mda.loc[tuple,new_cols]
pos_sampling.to_csv('all_time_industry_money_til2021.csv',encoding='gb18030')

In [329]:
'''
正负样本配对,将同一行业，同一年份的所有公司的资产排序。
'''
all_firm = pd.read_csv('all_time_industry_money_til2021.csv',encoding='gb18030').set_index(['Stkcd','EndDate'])
ind_asset_dict = {}
for firm, year in all_firm.index:
    ind = all_firm.loc[(firm,year),'Industry']
    asset = all_firm.loc[(firm,year),'TotalAssets']
    if (ind,year) not in ind_asset_dict.keys():
        ind_asset_dict[(ind,year)]=[asset]
    else:
        ind_asset_dict[(ind,year)].append(asset)
    
for ind, year in ind_asset_dict.keys():
    temp_list = ind_asset_dict[(ind,year)]
    # 对列表排序
    ind_asset_dict[(ind,year)] = sorted(temp_list)

'''
当无法根据资产进行正负样本匹配时,采用total sales。将同一行业,同一年份的所有公司的total sales排序。
'''
ind_sales_dict = {}
for firm, year in all_firm.index:
    ind = all_firm.loc[(firm,year),'Industry']
    asset = all_firm.loc[(firm,year),'OperatingRevenue']
    
    if (ind,year) not in ind_sales_dict:
        ind_sales_dict[(ind,year)]=[asset]
    else:
        ind_sales_dict[(ind,year)].append(asset)
    
for ind, year in ind_sales_dict.keys():
    temp_list = ind_sales_dict[(ind,year)]
    # 对列表排序
    ind_sales_dict[(ind,year)] = sorted(temp_list)

ind_asset_dict, ind_sales_dict

({('综合', 2003): [715065177.42,
   803756491.81,
   861453817.15,
   1023054248.38,
   1105722534.28,
   1153618257.42,
   1274704955.23,
   1330440806.86,
   1461212597.18,
   2000514188.93,
   2437227899.69,
   2446933863.87,
   2649814350.27,
   3302970657.24,
   3371813800.45,
   3994024829.36,
   4490661372.85,
   4538444885.46,
   6045346360.33],
  ('综合', 2004): [755439029.56,
   1037425239.38,
   1038599049.31,
   1063458822.65,
   1162450392.79,
   1179134857.05,
   1330149688.17,
   1331932161.11,
   1350676753.01,
   1813337840.63,
   2302935990.54,
   2791407669.0,
   3039560212.96,
   3161639289.94,
   3871695338.61,
   4199200660.32,
   5595951425.77,
   5690476177.83,
   6304614787.55],
  ('综合', 2005): [507114901.37,
   859358659.13,
   915619182.0,
   952210847.61,
   1148526023.86,
   1210913898.17,
   1261447715.39,
   1298055688.06,
   1331292937.84,
   1475725127.81,
   1736234150.41,
   1792535811.46,
   1865826621.52,
   2912264951.41,
   2994460618.57,
   338692184

In [331]:
train_test = pd.DataFrame(columns = ['Stkcd','pred_year','time_series','label','industry','asset'])
train_test.set_index(['Stkcd','pred_year'],inplace=True)

def generate_series(year, times):
    series = [year]
    for t in times:
        if year > t and (year-t) <= 5:
            series.append(t)
    # 将 series 排序
    series = sorted(series,reverse=True)

    # 将不连续的部分截断
    #series = [series[i] for i in range(len(series)) if series[0]-series[i]==i]
    return series
    
def generate_interval(neg_asset, pool):
    cand_pool = []
    position = pool.index(neg_asset)
    if position+3 < len(pool) and abs(pool[position+3] - neg_asset)/neg_asset <= 0.3: # 判断资产的差是否在百分之三十以内
        cand_pool += [pool[position+1],pool[position+2],pool[position+3]]
    
    elif position+2 < len(pool) and abs(pool[position+2] - neg_asset)/neg_asset <= 0.3:
        cand_pool += [pool[position+1],pool[position+2]]
    
    elif position+1 < len(pool) and abs(pool[position+1] - neg_asset)/neg_asset <= 0.3:
        cand_pool += [pool[position+1]]
    
    if position-3 >= 0 and abs(pool[position-3] - neg_asset)/neg_asset <= 0.3:
        cand_pool += [pool[position-1],pool[position-2],pool[position-3]]

    elif position-2 >= 0 and abs(pool[position-2] - neg_asset)/neg_asset <= 0.3:
        cand_pool += [pool[position-1],pool[position-2]]

    elif position-1 >= 0 and abs(pool[position-1] - neg_asset)/neg_asset <= 0.3:
        cand_pool += [pool[position-1]]
        
    return cand_pool

random.seed(0)

series_by_firm = pd.read_csv('fraud_series_net.csv',encoding='gb18030').set_index(['Stkcd','year'])

# 列表记录已经被选择的firm, 不记录year了
used_firm_year = []

all_firm_index = all_firm.index

# asset_id: 一个以asset 为索引，id 为value的字典
asset_id = all_firm.reset_index(drop=False).loc[:,['Stkcd','TotalAssets']].set_index('TotalAssets',inplace=False)

for firm, year in series_by_firm.index:

    # 先生成负样本（欺诈样本）
    if series_by_firm.loc[(firm,year),'is_fraud']==False:
        continue
    elif (firm,year-5) in series_by_firm.index and series_by_firm.loc[(firm,year-5),'is_fraud']==True: #说明往前推5年都还是欺诈，即连续第六年欺诈，这样的样本去掉，因为没有非欺诈年份的数据
        continue
    
    # 生成正样本（非欺诈样本）
    neg_ind = series_by_firm.loc[(firm,year),'industry']
    neg_asset = series_by_firm.loc[(firm,year),'asset']
    # print(neg_ind, year)

    if (neg_ind, year) not in ind_asset_dict.keys():
        print('%d年没有%s行业的公司:' % (year, neg_ind))
        continue
    
    asset_pool = ind_asset_dict[(neg_ind, year)].copy()
    # 把欺诈样本的总资产插入到 同一年同一个行业的资产列表中进行排序
    asset_pool.append(neg_asset)
    sorted_pool = sorted(asset_pool)
    
    # p = sorted_pool.index(neg_asset)
    interval = generate_interval(neg_asset,sorted_pool)
    # print(neg_asset, asset_pool, interval)
    if len(interval)==0:
        is_found=False
        # print('%d年的公司%s没有匹配到非欺诈公司' % (year,firm))
        continue

    selected_asset = random.choice(interval)

    f = asset_id.loc[selected_asset,'Stkcd']
    is_found = True

    while (f,year-1) not in all_firm_index or (f, year) in used_firm_year:
        interval.remove(selected_asset)
        if len(interval)==0:
            is_found=False
            print('%d年的公司%s没有匹配到非欺诈公司' % (year,firm))
            break
        selected_asset = random.choice(interval)
        f = asset_id.loc[selected_asset,'Stkcd']
    
    if is_found:
        used_firm_year.append((f,year))
        train_test.loc[(firm,year),'time_series'] = json.dumps(generate_series(year, series_by_firm.loc[firm,:].index.to_list()))
        train_test.loc[(firm,year),'label'] = 1
        train_test.loc[(firm,year),['industry','asset']] = [neg_ind, neg_asset]

        train_test.loc[(f,year),'time_series'] = json.dumps(generate_series(year, all_firm.loc[f,:].index.to_list()))
        train_test.loc[(f,year),'label'] = 0
        train_test.loc[(f,year),['industry','asset']] = [neg_ind, selected_asset]
        # print(firm,year)
# train_test.to_csv('train_test.csv',encoding='gb18030')

2007年的公司593没有匹配到非欺诈公司
2018年的公司668没有匹配到非欺诈公司
2017年的公司995没有匹配到非欺诈公司
2017年的公司996没有匹配到非欺诈公司
2012年的公司2180没有匹配到非欺诈公司
2017年的公司2188没有匹配到非欺诈公司
2012年的公司2213没有匹配到非欺诈公司
2019年的公司2325没有匹配到非欺诈公司
2017年的公司2473没有匹配到非欺诈公司
2015年的公司2473没有匹配到非欺诈公司
2016年的公司2637没有匹配到非欺诈公司
2017年的公司2683没有匹配到非欺诈公司
2017年的公司300277没有匹配到非欺诈公司
2020年的公司300598没有匹配到非欺诈公司
2018年的公司600086没有匹配到非欺诈公司
2014年的公司600108没有匹配到非欺诈公司
2016年的公司600234没有匹配到非欺诈公司
2015年的公司600234没有匹配到非欺诈公司
2016年的公司600313没有匹配到非欺诈公司
2020年的公司600354没有匹配到非欺诈公司
2016年的公司600436没有匹配到非欺诈公司
2019年的公司600467没有匹配到非欺诈公司
2013年的公司600477没有匹配到非欺诈公司
2016年的公司600552没有匹配到非欺诈公司
2018年的公司600586没有匹配到非欺诈公司
2011年的公司600624没有匹配到非欺诈公司
2012年的公司600747没有匹配到非欺诈公司
2016年的公司600777没有匹配到非欺诈公司
2015年的公司600807没有匹配到非欺诈公司
2018年的公司601113没有匹配到非欺诈公司
2016年的公司601258没有匹配到非欺诈公司
2018年的公司603508没有匹配到非欺诈公司
2018年的公司603778没有匹配到非欺诈公司
2017年的公司603778没有匹配到非欺诈公司
2018年的公司603799没有匹配到非欺诈公司


In [332]:
train_test.to_csv('train_test.csv',encoding='gb18030')

In [333]:
# train_test数据集的一些特征
train_test = pd.read_csv('train_test.csv',encoding='gb18030')
split_by_label = train_test.reset_index(drop=False).set_index(['label'],inplace=False)
fraud = split_by_label.loc[1,:].Stkcd
non_fraud = split_by_label.loc[0,:].Stkcd
print(fraud.shape, non_fraud.shape)
print(len(set(fraud)),len(set(non_fraud)))
'''
(1702,) (1702,)
897 773
'''

(2865,) (2865,)
1330 1292


'\n(1702,) (1702,)\n897 773\n'

In [23]:
train_test = pd.read_csv('train_test.csv',encoding='gb18030')
symbol_list = set(train_test.Stkcd)
query_by_Stkcd = pd.DataFrame(columns=['code'])
# 把有 isin 码的 symbol 筛出来，用于国泰安的查询
query_by_Stkcd['code'] = [str(x).zfill(6) for x in symbol_list]
query_by_Stkcd.sort_values(by="code", inplace=True, ascending=True) 
query_by_Stkcd.to_excel('code_for_CSMAR.xlsx',index=None, )
# code_for_CSMAR.xlsx用于从国泰安下载关系数据

### 至此，欺诈/非欺诈公司的匹配已完成，接下来是根据数据集中的公司去生成Corporate KG

In [335]:
# 高管关系
gaoguan = pd.read_excel('高管关系/CG_Director_raw.xlsx',encoding='gb18030')
gaoguan.drop([0,1],axis=0,inplace=True)
gaoguan = gaoguan.reset_index(drop=True)

In [4]:
train_test = pd.read_csv('train_test.csv',encoding='gb18030')
all_firms=set(train_test.Stkcd)
train_test.shape
len(all_firms)

2622

In [5]:
# 将上市公司全称和股票代码对应起来:
stk_map_name_id = pd.DataFrame(columns=['FullName','Stkcd'])

name_id_dict = {}
corp_basic_info = pd.read_excel('上市公司基本情况/STK_LISTEDCOINFOANL.xlsx',encoding='gb18030')
corp_basic_info.drop([0,1],inplace=True)
for i in corp_basic_info.index:
    key = corp_basic_info.loc[i,'FullName']
    value = corp_basic_info.loc[i,'Symbol']
    name_id_dict[key] = value

stk_map_name_id.FullName = name_id_dict.keys()
stk_map_name_id.Stkcd = name_id_dict.values()
# stk_map_name_id.to_csv('stk_fullname.csv',encoding='gb18030',index=None)

In [338]:
gaoguan_new = pd.DataFrame(columns=['Stkcd','manager','manager_id','position','interval'])

gaoguan_new.to_csv('高管关系/gaoguan.csv',encoding='gb18030',index=None)
# gaoguan_new.set_index(['Stkcd','manager'],inplace=True)

for i in gaoguan.index:
    firm = gaoguan.loc[i,'Stkcd']
    manager = gaoguan.loc[i,'D0101b']
    PersonID = gaoguan.loc[i,'PersonID']
    if int(firm) not in all_firms:
        continue
    year = gaoguan.loc[i,'Reptdt'].split('-')[0]
    temp = pd.DataFrame(
        {'Stkcd':firm,
        'manager':manager,
        'manager_id':PersonID,
        'position':gaoguan.loc[i,'D0201b'],
        'interval':year
        },index=[0])
    temp.to_csv('高管关系/gaoguan.csv',encoding='gb18030',mode='a',header=None,index=None)


In [84]:
# 供应商关系, 暂时不需要
'''gongying = pd.read_excel('前五大供应商关系/SC_TopFivePurchaseInfo_raw.xlsx',encoding='gb18030')
gongying.drop([0,1],axis=0,inplace=True)
gongying = gongying.reset_index(drop=True)
# gongying.set_index(['Symbol'],inplace=True)
gongying.set_index(['InstitutionName'],inplace=True)
print(gongying.shape)

# 先去掉一些看不出来公司名字的
import encodings
rule = r'第*[一二三四五A-Z]*供[应货方商]-*[一二三四五六七八九十a-eA-Z0-9]*|[A-E甲乙丙丁戊]+[有限]*公司|公司[一二三四五0-9]+|合计|第[一二三四五0-9]+[名位]|单位-*[0-9A-Z一二三四五]+|法人[一二三四五]*|客户[一二三四五0-9]*|排名[1-5]*|品牌[0-9A-Z]*|医药商[一二三四五]*|企业[A-Z]+'
drop_index = set()
for index in set(gongying.index):
    if len(re.findall(rule,index))>0:
        drop_index.add(index)
    elif len(index)==1 and len(re.findall(r'[0-9A-Z甲乙丙丁戊]',index))>0: # 名字就叫 A,B,C...1,2,3,4...
        drop_index.add(index)
    elif (len(index)>=1 or len(index)<=2) and len(re.findall(r'[A-Z][0-9]',index))>0: #A1,B2
        drop_index.add(index)
    elif '*' in index or '某公司' in index or '同一控制下' in index: # 曹**, ****技术有限公司
        drop_index.add(index)

gongying2 = gongying.drop(list(drop_index),axis=0,inplace=False)
gongying2.to_csv('前五大供应商关系/SC_TopFivePurchaseInfo.csv',encoding='gb18030')

# 生成用于组成KG的txt文件
gongying_temp = pd.DataFrame(columns=['Stkcd','gongying_firm','interval'])

gongying_temp.to_csv('前五大供应商关系/gongying.csv',encoding='gb18030')

gongying2 = pd.read_csv('前五大供应商关系/SC_TopFivePurchaseInfo.csv',encoding='gb18030')
for i in gongying2.index:
    symbol = gongying2.loc[i,'Symbol']
    year = int(gongying2.loc[i,'EndDate'].split('/')[0])
    if not gongying2.loc[i,'BusinessSymbol'] is np.nan:
        gongying_firm = gongying2.loc[i,'BusinessSymbol'].split(';')[0]
    else:
        gongying_firm = gongying2.loc[i,'InstitutionName']
    gongying_temp = pd.DataFrame(
        {'Stkcd':symbol,
        'gongying_firm':gongying_firm,
        'interval':year},index=[0])
    gongying_temp.to_csv('前五大供应商关系/gongying.csv',encoding='gb18030',mode='a',header=None,index=None)'''

(118340, 10)


In [339]:
# 亲属关系
qinshu = pd.read_excel('股权及亲属关系/HLD_Shrrelchain.xlsx',encoding='gb18030')
qinshu.drop([0,1],inplace=True)
qinshu = qinshu.reset_index(drop=True)

In [319]:
len(set(qinshu.Stkcd))

1670

In [340]:
# 把所有个体人名记下来，在收集股东关系的时候把人名往外扩
pattern = r'近*亲属|配偶的*|夫妻|[父母][子女亲]|[堂亲长胞]*[兄弟姐妹]+的*|[叔侄女儿子]+'
people = set()
qinshu_temp = pd.DataFrame(columns=['person1','person2','interval','detail','related_stk'])
qinshu_temp.to_csv('股权及亲属关系/qinshu.csv',encoding='gb18030',index=None)

for i in qinshu.index:
    reltype = qinshu.loc[i,'S0601a']
    detail = qinshu.loc[i,'S0607a']
    if reltype==6 and detail is not np.nan:#reltype=6为亲属关系
        year = int(qinshu.loc[i,'Reptdt'].split('-')[0])
        person1 = qinshu.loc[i,'S0603a']
        person2 = qinshu.loc[i,'S0604a']
        keyword = re.findall(pattern,detail.replace('女士',''))
        if len(keyword)>0:
            people.add(person1)
            people.add(person2)
            qinshu_temp = pd.DataFrame(
                        {'person1':person1,
                        'person2':person2,
                        'interval':year,
                        'detail':detail,
                        'related_stk': qinshu.loc[i,'Stkcd']},index=[0])
                        # json.dumps(keyword,ensure_ascii=False)
            qinshu_temp.to_csv('股权及亲属关系/qinshu.csv',encoding='gb18030',mode='a',header=None,index=None)
# re.findall(pattern,'潮州合众、潮州市炎城策划咨询有限公司的股东之一李湘娟为王建瑜的兄长的配偶'.replace('女士',''))

In [341]:
# 提取股权及亲属关系文件中的股东关系
gudong_temp = pd.DataFrame(columns=['holded','holder','interval','detail'])
gudong_temp.to_csv('股权及亲属关系/gudong.csv',encoding='gb18030',index=None)

#这里的关联只是关联方关系，并不形成关联交易
'''guanlian_temp = pd.DataFrame(columns=['party1','party2','interval','detail'])
guanlian_temp.to_csv('股权及亲属关系/guanlian.csv',encoding='gb18030')'''

for i in qinshu.index:
    Stkcd = qinshu.loc[i,'Stkcd']
    year = int(qinshu.loc[i,'Reptdt'].split('-')[0])
    reltype = qinshu.loc[i,'S0601a']
    is_holded_stk_gudong = qinshu.loc[i,'S0602a']
    person1 = qinshu.loc[i,'S0603a']
    person2 = qinshu.loc[i,'S0604a']
    if is_holded_stk_gudong==1:
        gudong_temp = pd.DataFrame(
            {'holded':Stkcd,
            'holder':person1,
            'interval':year},index=[0])
        gudong_temp.to_csv('股权及亲属关系/gudong.csv',encoding='gb18030',mode='a',header=None,index=None)

    is_holder_stk_gudong = qinshu.loc[i,'S0605a']
    if is_holder_stk_gudong==1:
        gudong_temp = pd.DataFrame(
            {'holded':Stkcd,
            'holder':person2,
            'interval':year},index=[0])
        gudong_temp.to_csv('股权及亲属关系/gudong.csv',encoding='gb18030',mode='a',header=None,index=None)

    if reltype==1: # 持股关系
        detail = qinshu.loc[i,'S0607a']
        gudong_temp = pd.DataFrame(
            {'holded':person1,
            'holder':person2,
            'interval':year,
            'detail':detail},index=[0])
        gudong_temp.to_csv('股权及亲属关系/gudong.csv',encoding='gb18030',mode='a',header=None,index=None)
    '''elif reltype==2: # 关联关系
        detail = qinshu.loc[i,'S0607a']
        guanlian_temp = pd.DataFrame(
            {'party1':person1,
            'party2':person2,
            'interval':year,
            'detail':detail},index=[0])
        guanlian_temp.to_csv('股权及亲属关系/guanlian.csv',encoding='gb18030',mode='a',header=None,index=None)'''

# 去重
gudong_temp = pd.read_csv('股权及亲属关系/gudong.csv',encoding='gb18030')
gudong_temp.drop_duplicates(subset=['holded','holder','interval'],keep='first',inplace=True)
gudong_temp.to_csv('股权及亲属关系/gudong.csv',encoding='gb18030',index=None)


In [343]:
# 子公司关系
subfirm = pd.read_excel('上市公司子公司情况表/FN_Fn061.xlsx',encoding='gb18030')
subfirm.drop([0,1],inplace=True)
subfirm = subfirm.reset_index(drop=True)

In [347]:
sub_temp = pd.DataFrame(columns=['Stkcd','subsidiary','interval'])
sub_temp.to_csv('上市公司子公司情况表/subsidiary.csv',encoding='gb18030',index=None)
for i in subfirm.index:
    year = int(subfirm.loc[i,'EndDate'].split('-')[0])
    if year < 2003:
        continue
    sub_temp = pd.DataFrame(
            {'Stkcd':subfirm.loc[i,'Stkcd'],
            'subsidiary':subfirm.loc[i,'FN_Fn06101'],
            'interval':year},index=[0])
    sub_temp.to_csv('上市公司子公司情况表/subsidiary.csv',encoding='gb18030',mode='a',header=None,index=None)

In [348]:
subfirm = pd.read_csv('上市公司子公司情况表/subsidiary.csv',encoding='gb18030')
subfirm.drop_duplicates(subset=['Stkcd','subsidiary','interval'],keep='first',inplace=True)
subfirm = subfirm.reset_index(drop=True)

subfirm_temp = pd.DataFrame(columns=['Stkcd','subsidiary','interval'])
subfirm_temp.to_csv('上市公司子公司情况表/subsidiary_1st_filter.csv',encoding='gb18030',index=None)

for i in subfirm.index:
    sub = subfirm.loc[i,'subsidiary']
    if 'Inc' in sub and 'Inc.' not in sub:
        sub = sub.replace('Inc','Inc.')
    elif 'INC' in sub and 'INC.' not in sub:
        sub = sub.replace('INC','INC.')
    
    if 'Ltd' in sub and 'Ltd.' not in sub:
        sub = sub.replace('Ltd','Ltd.')
    elif 'LTD' in sub and 'LTD.' not in sub:
        sub = sub.replace('LTD','LTD.')
    
    if 'Corp' in sub and 'Corp.' not in sub:
        sub = sub.replace('Corp','Corp.')
    elif 'CORP' in sub and 'CORP.' not in sub:
        sub = sub.replace('CORP','CORP.')
    
    if 'Pte' in sub and 'Pte.' not in sub:
        sub = sub.replace('Pte','Pte.')
    elif 'PTE' in sub and 'PTE.' not in sub:
        sub = sub.replace('PTE','PTE.')

    sub = sub.replace(' ','')
    subfirm_temp = pd.DataFrame({'Stkcd':subfirm.loc[i,'Stkcd'],
                    'subsidiary':sub,
                    'interval':subfirm.loc[i,'interval']},index=[0])
    
    subfirm_temp.to_csv('上市公司子公司情况表/subsidiary_1st_filter.csv',encoding='gb18030',index=None,header=None,mode='a')

In [349]:
# 第三轮：把其中的上市公司（全名）转化为股票代码
subfirm = pd.read_csv('上市公司子公司情况表/subsidiary_1st_filter.csv',encoding='gb18030')
for i in subfirm.index:
    name = subfirm.loc[i,'subsidiary']
    if name in name_id_dict.keys():
        subfirm.loc[i,'subsidiary'] = name_id_dict[name]
subfirm.to_csv('上市公司子公司情况表/subsidiary_fullname_matched.csv',encoding='gb18030',index=None)

### 关联交易关系

In [351]:
guanlian_1 = pd.read_excel('关联方交易/RPT_Operation.xlsx',encoding='gb18030')
guanlian_2 = pd.read_excel('关联方交易/RPT_Operation1.xlsx',encoding='gb18030')

guanlian_1.drop([0,1],inplace=True)
guanlian_2.drop([0,1],inplace=True)

guanlian = pd.concat([guanlian_1,guanlian_2])
guanlian = guanlian.reset_index(drop=True)

In [353]:
guanlian_temp = pd.DataFrame(columns=['seller','buyer','interval','trans_type'])
guanlian_temp.to_csv('关联方交易/guanlian.csv',encoding='gb18030',index=None)

buy_pattern = r'[支代]付|购|信托产品|接受担保|接受[^A-Za-z0-9]*[劳服业务管理工程]+|承租|拆入|^[^发提]*[接受委托]*[贷借]款'
sell_pattern = r'[销出]售|转让|[提供]*担保|[代]*收[取]*|提供[^A-Za-z0-9]*[劳服业务管理工程]+|出租|拆[出借]+|承包|^[^委]*借款|[发放提供][贷借]款'
bi_pattern = r'互[相销]|共同'
useless_pattern = r'增资|技术服务|租赁|工程收入|资金往来|委托[经营利息]*|资产托管|还款|承销|资产置换|[劳务股权][交易托管]|装修|签署|抵款|抵押'

collected = []
break_point = 0

for i in guanlian.index: 
    if i < break_point:
        continue
    # 交易日期，不是公告日期
    year = int(guanlian.loc[i,'Trddt'].split('-')[0])
    
    # 交易性质：1-该交易为本公司与关联方之间的交易;2-关联公司之间的交易
    trans_nature = guanlian.loc[i,'Trasub']
    
    # 交易方向：1=上市公司处于卖方立场，2=上市公司处于买方立场，3=上市公司处于无法分辨。
    trans_direct = guanlian.loc[i,'Direction']
    content = guanlian.loc[i,'Content']
    trans_type = guanlian.loc[i,'Kind']

    Stkcd = int(guanlian.loc[i,'Stkcd'])
    guanlian_firm = guanlian.loc[i,'Repart'].split(',')
    if trans_nature==1:
        if trans_direct == 1:
            seller,buyer = Stkcd, guanlian_firm[0]
        elif trans_direct==2:
            seller,buyer = guanlian_firm[0], Stkcd
        else: # trans_direct=3 多为共同执行
            seller,buyer = guanlian_firm[0], Stkcd

    elif trans_nature==2:
        #print(trans_type)
        if len(re.findall(buy_pattern, trans_type))>0:
            seller, buyer = guanlian_firm[-1], guanlian_firm[0]
        
        elif len(re.findall(sell_pattern, trans_type)) > 0:
            seller, buyer = guanlian_firm[0],guanlian_firm[-1]
        
        elif len(re.findall(bi_pattern, trans_type)) > 0: # 双方共同执行
            seller, buyer = guanlian_firm[0],guanlian_firm[-1]

        elif '供' in trans_type:
            try:
                firm_position = [content.find(j) for j in guanlian_firm]
            except:
                continue
            if firm_position[0] <= firm_position[-1]:
                seller, buyer = guanlian_firm[0],guanlian_firm[-1]
            else:
                seller, buyer = guanlian_firm[-1],guanlian_firm[0]
        elif len(re.findall(useless_pattern, trans_type)) > 0:
            continue
        else: # 其他pattern太多了，涵盖不完了
            continue
    else:
        continue
                
    if (seller, buyer, year) in collected:
        continue
    
    guanlian_temp = pd.DataFrame({'seller':seller,
                'buyer':buyer,
                'interval':year,
                'trans_type':trans_type},index=[0])
    guanlian_temp.to_csv('关联方交易/guanlian.csv',encoding='gb18030',mode='a',header=None,index=None)
    collected.append((seller, buyer, year))
    del seller, buyer, year
    # 要跑三个小时


In [2]:
# 第二轮筛选，排除买/卖方指代不明的关联交易
guanlian = pd.read_csv('关联方交易/guanlian.csv',encoding='gb18030')
print(guanlian.shape)

firm_except = r'[高管理]+人员|公司管理层|骨干|成员|员工|关联方|[董|监]事|其他|关联公司|薪酬|副总|[合联营国有关联]+企业|集团内企业|总工程师|总经理|[0-9]*家[分子项目]+公司|[名个位][自然一致行动发起]+人|[名个位]+股东'
drop_list = []
useless_name = 'ltd. .ltd inc. limitedsp.zo.o. s.l. llc. s.a. 子公司 limited股东'

for i in guanlian.index:
    seller = guanlian.loc[i,'seller']
    buyer = guanlian.loc[i,'buyer']
    if type(seller)==str:
        if '等' in seller and len(re.findall(r'[高中]等|等级',seller)) == 0:
            guanlian.loc[i,'seller'] = seller.split('等')[0]
        elif '及' in seller:
            guanlian.loc[i,'seller'] = seller.split('及')[0]

        if len(re.findall(firm_except,seller))>0 or seller.lower() in useless_name:
            drop_list.append(i)

    if type(buyer)==str:
        if '等' in buyer and len(re.findall(r'[高中]等|等级',buyer))== 0:
            guanlian.loc[i,'buyer'] = buyer.split('等')[0]
        elif '及' in buyer:
            guanlian.loc[i,'buyer'] = buyer.split('及')[0]

        if len(re.findall(firm_except,buyer))>0 or buyer.lower() in useless_name:
            drop_list.append(i)

guanlian.drop(drop_list,inplace=True)
guanlian.to_csv('关联方交易/guanlian_second_filter.csv',encoding='gb18030',index=None)

(556263, 4)


In [6]:
# 第三轮：把其中的上市公司（全名）转化为股票代码
guanlian = pd.read_csv('关联方交易/guanlian_second_filter.csv',encoding='gb18030')
for i in guanlian.index:
    for j in range(2):
        name = guanlian.iloc[i,j]
        if name in name_id_dict.keys():
            guanlian.iloc[i,j] = name_id_dict[name]
            
guanlian.to_csv('关联方交易/guanlian_fullname_matched.csv',encoding='gb18030',index=None)

### 股东关系


In [44]:
shareholder1 = pd.read_excel('十大流通股股东/HLD_Negshr_2003_2006.xlsx',encoding='gb18030')
shareholder2 = pd.read_excel('十大流通股股东/HLD_Negshr_2007_2011.xlsx',encoding='gb18030')
shareholder3 = pd.read_excel('十大流通股股东/HLD_Negshr_2012_2016.xlsx',encoding='gb18030')
shareholder4 = pd.read_excel('十大流通股股东/HLD_Negshr_2017_2021.xlsx',encoding='gb18030')
shareholder1.drop([0,1],inplace=True)
shareholder2.drop([0,1],inplace=True)
shareholder3.drop([0,1],inplace=True)
shareholder4.drop([0,1],inplace=True)
shareholder = pd.concat([shareholder1,shareholder2,shareholder3,shareholder4])
shareholder = shareholder.reset_index(drop=True)

In [45]:
gudong_temp = pd.DataFrame(columns=['holded','holder','interval','detail'])
gudong_temp.to_csv('十大流通股股东/top10_gudong.txt',encoding='gb18030',index=None)
for i in shareholder.index:
    year = shareholder.loc[i,'ChangeStartDate']
    if year is np.nan:
        continue
    else:
        year = year.split('-')[0]
    Stkcd = shareholder.loc[i,'Stkcd']
    gudong = shareholder.loc[i,'S0401a']
    gudong_temp = pd.DataFrame({'holded':Stkcd,
                'holder':gudong,
                'interval':year},index=[0])
    gudong_temp.to_csv('十大流通股股东/top10_gudong.txt',encoding='gb18030',mode='a',header=None,index=None)

# 生成top10_gudong.txt之后，手动把里面的-换成-，（）换成()

In [46]:
gudong1 = pd.read_csv('十大流通股股东/top10_gudong.txt',encoding='gb18030')
# 手动把gudong.csv里面的半角—换成-，（）换成()
gudong2 = pd.read_csv('股权及亲属关系/gudong.csv',encoding='gb18030')

print(gudong1.shape, gudong2.shape)
gudong = pd.concat([gudong1,gudong2],ignore_index=True)
#gudong.drop_duplicates(subset=['holded','holder','interval'],keep='first',inplace=True)
gudong = gudong.reset_index(drop=True)
gudong.to_csv('十大流通股股东/gudong_full.txt',encoding='gb18030',index=None)

(938200, 4) (164292, 4)


In [47]:
# 二轮筛选
gudong = pd.read_csv('十大流通股股东/gudong_full.txt',encoding='gb18030')
print(gudong.shape)
gudong_temp = pd.DataFrame(columns=['holded','holder','interval','detail'])

# 已经超过了100万，暂时存到csv里（不存txt了，浏览起来不方便），下一步去重之后就没有100万了
gudong_temp.to_csv('十大流通股股东/gudong_fullname_matched.csv',encoding='gb18030',index=None)

firm_except = r'[高级管理]+人员|骨干|成员|其他|[名个位][自然一致行动发起]+人|[名个位]+股东'

for i in gudong.index:
    holded = gudong.loc[i,'holded']
    holder = gudong.loc[i,'holder']
    
    if type(holded)==str:
        if '等' in holded and len(re.findall(r'[高中]等|等[级权]',holded)) == 0:
            holded = holded.split('等')[0]
        elif '及' in holded:
            holded = holded.split('及')[0]

        if len(re.findall(firm_except,holded))>0:
            continue
        if holded.count('-')>=1: 
            # 把带-的基金产品截取其中的公司
            holded = holded.split('-')[0]
        elif holded in name_id_dict.keys():
            # 将股东、关联关系里的公司全名转换成股票代码
            holded = name_id_dict[holded]
    
    if type(holder)==str:
        if '等' in holder and len(re.findall(r'[高中]等|等[级权]',holder)) == 0:
            holder = holder.split('等')[0]
        elif '及' in holder:
            holder = holder.split('及')[0]
        if len(re.findall(firm_except,holder))>0:
            continue
        if holder.count('-')>=1: 
            # 把带-的基金产品截取其中的公司
            holder = holder.split('-')[0]
        elif holder in name_id_dict.keys():
            # 将股东、关联关系里的公司全名转换成股票代码
            holder = name_id_dict[holder]

    gudong_temp = pd.DataFrame({'holded':holded,
                'holder':holder,
                'interval':gudong.loc[i,'interval'],
                'detail':gudong.loc[i,'detail']},index=[0])
    gudong_temp.to_csv('十大流通股股东/gudong_fullname_matched.csv',encoding='gb18030',index=None,mode='a',header=None)
    

(1102492, 4)


In [48]:
# 股东关系去重
gudong_temp = pd.read_csv('十大流通股股东/gudong_fullname_matched.csv',encoding='gb18030')
print(gudong_temp.shape)

gudong_temp.drop_duplicates(subset=['holded','holder','interval'],keep='first',inplace=True)
print(gudong_temp.shape)

gudong_temp.to_csv('十大流通股股东/gudong_fullname_matched.csv',encoding='gb18030',index=None)

(1102017, 4)
(544695, 4)


## 把重复的人名后面加上gaoguan.csv中的manager_id，以区分重复的人名

In [30]:
gaoguan = pd.read_csv('高管关系/gaoguan.csv',encoding='gb18030')
people_id = {}
repeat_names = set()
for i in gaoguan.index:
    name = gaoguan.loc[i,'manager']
    id = gaoguan.loc[i,'manager_id']
    if name in people_id.keys():
        if id not in people_id[name]:
            people_id[name].append(gaoguan.loc[i,'manager_id'])
            repeat_names.add(name)
    else:
        people_id[name] = [gaoguan.loc[i,'manager_id']]
print(len(people_id),len(repeat_names))

# 构造一个字典中的字典：name：stk：id，这个字典用于在扫描所有关系中的重名人，在人名后面加上-id
duel_people_stk_id = {}
for i in gaoguan.index:
    name = gaoguan.loc[i,'manager']
    stk = gaoguan.loc[i,'Stkcd']
    id = gaoguan.loc[i,'manager_id']
    if name in repeat_names:
        # 人名重复
        if name not in duel_people_stk_id:
            duel_people_stk_id[name] = {}
        #if stk not in people_stk_id[name]:
        duel_people_stk_id[name][stk] = id
        

89863 8737


In [367]:
# 高管关系人名配对
gaoguan_temp = pd.DataFrame(columns=['Stkcd','manager','manager_id','position','interval'])
gaoguan_temp.to_csv('高管关系/gaoguan_no_dup_person.csv',index = None, encoding='gb18030')

for i in gaoguan.index:
    name = gaoguan.loc[i,'manager']
    stk = gaoguan.loc[i,'Stkcd']
    if name in repeat_names:
        # 人名重复
        name = name + '-' + str(duel_people_stk_id[name][stk])
        # 这一步最耗时间，因为要修改dataframe中的值 gaoguan.loc[i,'manager'] = new_name
    gaoguan_temp = pd.DataFrame({
        'Stkcd': stk,
        'manager': name,
        'manager_id': gaoguan.loc[i,'manager_id'],
        'position': gaoguan.loc[i,'position'],
        'interval': gaoguan.loc[i,'interval'],
    },index=[0])
    gaoguan_temp.to_csv('高管关系/gaoguan_no_dup_person.csv',index = None, header=None, mode='a',encoding='gb18030')


In [49]:
# 股东的人名配对率很低，股东里面的很多（人，股票）都和高管里面的对不上。
gudong = pd.read_csv('十大流通股股东/gudong_fullname_matched.csv',encoding='gb18030')
print(gudong.shape)

(544695, 4)


In [50]:
gudong_temp = pd.DataFrame(columns=['holded','holder','interval','detail'])
gudong_temp.to_csv('十大流通股股东/gudong_no_dup_person.csv',index=None,encoding='gb18030')

for i in gudong.index:
    holder = gudong.loc[i,'holder']
    holded = gudong.loc[i,'holded']
    # 股东关系里，人名只可能出现在holder里面
    if type(holder)==str and len(holder)<=4 and holder in repeat_names: # 保证holder是一个人
        if type(holded)==int:
            if holded in duel_people_stk_id[holder]:
                holder = holder + '-' + str(duel_people_stk_id[holder][holded])
        elif type(gudong.loc[i+1,'holder'])==int:
            holded = gudong.loc[i+1,'holded'] #往后找一个
            if holded in duel_people_stk_id[holder]:
                holder = holder + '-' + str(duel_people_stk_id[holder][holded])
        elif type(gudong.loc[i-1,'holder'])==int:
            holded = gudong.loc[i-1,'holded'] #往前找一个
            if holded in duel_people_stk_id[holder]:
                holder = holder + '-' + str(duel_people_stk_id[holder][holded])
        elif type(gudong.loc[i+2,'holder'])==int:
            holded = gudong.loc[i+1,'holded'] #往后找两个
            if holded in duel_people_stk_id[holder]:
                holder = holder + '-' + str(duel_people_stk_id[holder][holded])
        elif type(gudong.loc[i-2,'holder'])==int:
            holded = gudong.loc[i-1,'holded'] #往前找两个
            if holded in duel_people_stk_id[holder]:
                holder = holder + '-' + str(duel_people_stk_id[holder][holded])
    gudong_temp = pd.DataFrame({
        'holded': gudong.loc[i,'holded'],
        'holder': holder,
        'interval': gudong.loc[i,'interval'],
        'detail': gudong.loc[i,'detail']},index=[0])
    gudong_temp.to_csv('十大流通股股东/gudong_no_dup_person.csv',index=None,encoding='gb18030',header=None, mode='a')

In [51]:
# 在人名去重之后，剔除金融公司类别的大股东
gudong = pd.read_csv('十大流通股股东/gudong_no_dup_person.csv',encoding='gb18030')
drop_index = []
fin_pattern = r'基金|证券|银行|保险|[中农工建]行|财富|国泰君安|申银万国|国际金融|资管|易方达|资产管理|信托|理财'
for i in gudong.index:
    holder = gudong.loc[i,'holder']
    if type(holder)==str and len(re.findall(fin_pattern,holder))>0:
            drop_index.append(i)
gudong.drop(drop_index,inplace=True)
gudong.to_csv('十大流通股股东/gudong_not_fin.csv',encoding='gb18030',index=None)

In [13]:
# 亲戚关系
# '李中秋','吉祥','王萍','王玲玲','刘建民','陈小兵','刘畅','李巍','赵晓红','胡颖', 这些亲戚文件中的人在高管关系中是有重复的
print(duel_people_stk_id['李中秋'])
#people_stk_id
qinshu = pd.read_csv('股权及亲属关系/qinshu.csv',encoding='gb18030')
qinshu_temp = pd.DataFrame(columns=['person1','person2','interval','detail','related_stk'])
qinshu_temp.to_csv('股权及亲属关系/qinshu_no_dup_person.csv',encoding='gb18030',index=None)
for i in qinshu.index:
    p1 = qinshu.loc[i,'person1']
    p2 = qinshu.loc[i,'person2']
    stk = qinshu.loc[i,'related_stk']
    if p1 in repeat_names and stk in duel_people_stk_id[p1]:
        p1 = p1 + '-' + str(duel_people_stk_id[p1][stk])
    if p2 in repeat_names and stk in duel_people_stk_id[p2]:
        p2 = p2 + '-' + str(duel_people_stk_id[p2][stk])
    qinshu_temp = pd.DataFrame({
        'person1':p1,
        'person2':p2,
        'interval':qinshu.loc[i,'interval'],
        'detail':qinshu.loc[i,'detail'],
        'related_stk':stk},index=[0])
    qinshu_temp.to_csv('股权及亲属关系/qinshu_no_dup_person.csv',encoding='gb18030',index=None, header=None, mode='a')

{20: 3079716, 600295: 30403809}


In [14]:
# 关联交易
guanlian = pd.read_csv('关联方交易/guanlian_fullname_matched.csv',encoding='gb18030')
guanlian_temp = pd.DataFrame(columns=['seller','buyer','interval','trans_type'])
guanlian_temp.to_csv('关联方交易/guanlian_no_dup_person.csv',encoding='gb18030',index=None)
for i in guanlian.index:
    seller, buyer = guanlian.loc[i,'seller'], guanlian.loc[i,'buyer']
    if type(buyer)==str and len(buyer)<=4 and buyer in repeat_names:
        if re.match(r'[0-9]+',seller) is not None:
            seller = int(seller)
        elif re.match(r'[0-9]+',guanlian.loc[i+1,'seller']) is not None:
            seller = int(guanlian.loc[i+1,'seller'])
        elif re.match(r'[0-9]+',guanlian.loc[i-1,'seller']) is not None:
            seller = int(guanlian.loc[i-1,'seller'])
        elif re.match(r'[0-9]+',guanlian.loc[i+1,'buyer']) is not None:
            seller = int(guanlian.loc[i+1,'buyer'])
        elif re.match(r'[0-9]+',guanlian.loc[i-1,'buyer']) is not None:
            seller = int(guanlian.loc[i-1,'buyer'])

        if seller in duel_people_stk_id[buyer]:
            buyer = buyer + '-' + str(duel_people_stk_id[buyer][seller])
        # 还原seller
        seller = guanlian.loc[i,'seller']
        
    elif type(seller)==str and len(seller)<=4 and seller in repeat_names:
        if re.match(r'[0-9]+',buyer) is not None:
            buyer = int(buyer)
        elif re.match(r'[0-9]+',guanlian.loc[i+1,'buyer']) is not None:
            buyer = int(guanlian.loc[i+1,'buyer'])
        elif re.match(r'[0-9]+',guanlian.loc[i-1,'buyer']) is not None:
            buyer = int(guanlian.loc[i-1,'buyer'])
        elif re.match(r'[0-9]+',guanlian.loc[i+1,'seller']) is not None:
            buyer = int(guanlian.loc[i+1,'seller'])
        elif re.match(r'[0-9]+',guanlian.loc[i-1,'seller']) is not None:
            buyer = int(guanlian.loc[i-1,'seller'])
        # print(seller, buyer, type(buyer), buyer in duel_people_stk_id[seller])
        if buyer in duel_people_stk_id[seller]:
            seller = seller + '-' + str(duel_people_stk_id[seller][buyer])
        buyer = guanlian.loc[i,'buyer']
    
    guanlian_temp = pd.DataFrame({
        'seller': seller,
        'buyer': buyer,
        'interval': guanlian.loc[i,'interval'],
        'trans_type': guanlian.loc[i,'trans_type']}, index=[0])
    guanlian_temp.to_csv('关联方交易/guanlian_no_dup_person.csv',encoding='gb18030',index=None, header=None, mode='a')

# 生成guanlian_no_dup_person.csv以后，手动将里面的半角（）替换为()

In [19]:
# 查看关联交易中存在哪些指代性的公司名字
guanlian = pd.read_csv('关联方交易/guanlian_fullname_matched.csv',encoding='gb18030')
buhege = set()
for i in guanlian.index:
    seller = guanlian.loc[i,'seller']
    buyer = guanlian.loc[i,'buyer']
    try:
        seller = int(seller)
    except:
        if len(seller) < 6:
            buhege.add(seller)
    try:
        buyer = int(buyer)
    except:
        if len(buyer) < 6:
            buhege.add(buyer)
with open('关联方交易/useless_list.json','w') as f:
    json.dump(list(buhege),f,ensure_ascii=False)

In [20]:
# 关联关系去重
guanlian = pd.read_csv('关联方交易/guanlian_no_dup_person.csv',encoding='gb18030')
print(guanlian.shape)
guanlian.drop_duplicates(subset=['seller','buyer','interval'],keep='first',inplace=True)
print(guanlian.shape)
# 没几个重复的，没有必要去重了

(532397, 4)
(530753, 4)


### 最后是生成用于输入pytorch框架的fin ratios格式

In [224]:
# 1.先把fraud_series_net.csv和 all_time_industry_money_til2021.csv里面的情感特征的缺失值补上，然后存到 NetDetect 文件夹下面的fraud_data.csv和nonfraud_data.csv里
neg_fin = pd.read_csv('fraud_series_net.csv', encoding='gb18030')
df = pd.DataFrame(columns=neg_fin.columns)
neg_fin.set_index('Stkcd',inplace=True)

df.to_csv('NetDetect/fraud_data.csv',encoding='gb18030',index=None)

for i in sorted(set(neg_fin.index)):
    df = neg_fin.loc[i,:].copy(deep=True)
    # 把dataframe做一个上下翻转，让时间成正序，和all_time_industry_money_til2021的一样
    df = df.iloc[::-1]
    if df[df.TextualSimilarity.isnull()==True].shape[0]==0:
        print(i,'no need to pad')
    elif df[df.TextualSimilarity.isnull()==True].shape[0] == df.shape[0]:
        df.iloc[:,16:] = np.zeros(shape=(df.shape[0],df.shape[1]-16))
    else:
        df_nan = df[df.TextualSimilarity.isnull()==True].iloc[:,16:]
        df_not_nan = df[df.TextualSimilarity.isnull()==False].iloc[0,16:]
        for i in range(df_nan.shape[0]):
            df.iloc[i,16:] = df_not_nan
    df.to_csv('NetDetect/fraud_data.csv',encoding='gb18030',header=None, mode='a')


'''
########################################################################################################################################################################
'''
pos_fin = pd.read_csv('all_time_industry_money_til2021.csv.csv', encoding='gb18030')
df = pd.DataFrame(columns=pos_fin.columns)
pos_fin.set_index('Stkcd',inplace=True)

df.to_csv('NetDetect/nonfraud_data.csv',encoding='gb18030',index=None)

for i in sorted(set(pos_fin.index)):
    df = pos_fin.loc[i,:].copy(deep=True)
    if df[df.TextualSimilarity.isnull()==True].shape[0]==0:
        print(i,'no need to pad')
    elif df[df.TextualSimilarity.isnull()==True].shape[0] == df.shape[0]:
        df.iloc[:,16:] = np.zeros(shape=(df.shape[0],df.shape[1]-16))
    else:
        df_nan = df[df.TextualSimilarity.isnull()==True].iloc[:,16:]
        df_not_nan = df[df.TextualSimilarity.isnull()==False].iloc[0,16:]
        for i in range(df_nan.shape[0]):
            df.iloc[i,16:] = df_not_nan

    df.to_csv('NetDetect/nonfraud_data.csv',encoding='gb18030',header=None, mode='a')


10 no need to pad
38 no need to pad
48 no need to pad
56 no need to pad
566 no need to pad
638 no need to pad
662 no need to pad
673 no need to pad
683 no need to pad
736 no need to pad
796 no need to pad
923 no need to pad
961 no need to pad
976 no need to pad
981 no need to pad
2006 no need to pad
2131 no need to pad
2160 no need to pad
2167 no need to pad
2183 no need to pad
2192 no need to pad
2196 no need to pad
2197 no need to pad
2217 no need to pad
2260 no need to pad
2278 no need to pad
2280 no need to pad
2309 no need to pad
2326 no need to pad
2334 no need to pad
2355 no need to pad
2382 no need to pad
2420 no need to pad
2432 no need to pad
2445 no need to pad
2456 no need to pad
2490 no need to pad
2505 no need to pad
2506 no need to pad
2512 no need to pad
2513 no need to pad
2514 no need to pad
2529 no need to pad
2537 no need to pad
2548 no need to pad
2568 no need to pad
2570 no need to pad
2574 no need to pad
2589 no need to pad
2602 no need to pad
2617 no need to pad

In [225]:
import torch
# 以公司为单位做标准化

pos_fin = pd.read_csv('NetDetect/nonfraud_data.csv', encoding='gb18030')
pos_fin.drop(['TotalAssets','Industry','OperatingRevenue'],axis=1,inplace=True)
pos_fin.rename({'EndDate':'year'},axis=1,inplace=True)

neg_fin = pd.read_csv('NetDetect/fraud_data.csv', encoding='gb18030')
neg_fin.drop(['is_fraud','industry','asset'],inplace=True,axis=1)

fin_ratios = pd.concat([neg_fin, pos_fin],ignore_index=True)

# IG (5处), DSIR (1处), RG (1处)
fin_ratios.IG = [1 if i==0 else i for i in fin_ratios.IG]
fin_ratios.DSIR = [1 if i==0 else i for i in fin_ratios.DSIR]
fin_ratios.RG = [1 if i==0 else i for i in fin_ratios.RG]

# 调整列的顺序, 拼接完以后，fin_ratios的列标是按字母大小排序，所以原来的指标会打乱
fin_ratios = fin_ratios[['Stkcd','year','AQI', 'AT', 'CFED', 'DSIR', 'DEPI', 'GMI', 'IG', 'LEV', 'OPM', 'RG', 'SG',
                     'SGEE', 'TextualSimilarity', 'PositiveVocabularyNum', 'NegativeVocabularyNum', 'TotalWordsNum',
                     'SentencesNum', 'WordsNum', 'EmotionTone1', 'EmotionTone2']]
columns = fin_ratios.columns
fin_ratios.set_index('Stkcd', inplace=True)

In [228]:
temp = pd.DataFrame(columns=columns)
temp.to_csv('findata_for_torch.csv', encoding='gb18030',index=None)

BN = torch.nn.BatchNorm1d(20)
for i in sorted(set(fin_ratios.index)):
    df = fin_ratios.loc[i,:]
    temp = df.copy(deep=True)
    normed_value = BN(torch.from_numpy(df.iloc[:,1:].values).float()).data.numpy()
    temp.iloc[:,1:] = normed_value
    temp.to_csv('findata_for_torch.csv',encoding='gb18030',header=None, mode='a')


In [221]:
df = fin_ratios.loc[4,:]
# df.set_index(['Stkcd','year'],inplace=True)
df

#normed_value = BN(torch.tensor(df.iloc[:,1:].values).float()).data.numpy()

,year,AQI,AT,CFED,DSIR,DEPI,GMI,IG,LEV,OPM,...,SG,SGEE,TextualSimilarity,PositiveVocabularyNum,NegativeVocabularyNum,TotalWordsNum,SentencesNum,WordsNum,EmotionTone1,EmotionTone2
Stkcd,,,,,,,,,,,,,,,,,,,,,
4,2013,1.404375,0.300574,-26839058.35,0.618852,0.849910,0.866284,1.664414,1.734157,0.079643,...,0.747556,1.158231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2012,0.332708,0.504699,9748495.69,1.370052,1.342407,1.117503,1.173889,0.788402,0.080258,...,1.306825,0.897442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2011,0.799126,0.379526,10172672.07,1.862770,1.241959,0.798826,0.971927,0.987812,0.123391,...,0.567295,1.683975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2010,1.726699,0.712908,-31938852.50,0.286495,0.975603,1.236623,0.641707,0.489171,0.153819,...,2.185922,0.506915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2009,0.701032,0.260640,42063512.92,0.990132,0.896783,0.930596,1.501864,1.241106,0.075441,...,1.387068,0.701708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2008,2.218927,0.258117,12912489.67,0.645264,0.900856,0.831692,1.417204,1.159388,-0.265133,...,1.070836,1.070409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2007,1.095075,0.238187,3049814.24,0.129920,0.832039,1.225166,1.372838,0.941762,-0.247270,...,0.886293,1.504313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2006,3.181621,0.223387,-12028455.85,1.104362,0.505974,0.832742,2.683214,1.255184,0.066942,...,0.743588,0.416767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2005,0.286208,0.393772,29638228.89,1.352339,2.393082,0.887890,0.324549,0.839016,-0.302678,...,0.574970,2.023285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [222]:
df1 = df.iloc[::-1]
df1

,year,AQI,AT,CFED,DSIR,DEPI,GMI,IG,LEV,OPM,...,SG,SGEE,TextualSimilarity,PositiveVocabularyNum,NegativeVocabularyNum,TotalWordsNum,SentencesNum,WordsNum,EmotionTone1,EmotionTone2
Stkcd,,,,,,,,,,,,,,,,,,,,,
4,2003,0.993801,0.502511,2322483.44,1.429028,0.920639,1.092874,1.247424,0.868941,0.063926,...,0.986054,0.940194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2004,1.090883,0.498192,3688228.02,1.245346,0.751103,1.219674,1.001391,0.929492,-0.030754,...,0.893025,1.085945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2005,0.286208,0.393772,29638228.89,1.352339,2.393082,0.887890,0.324549,0.839016,-0.302678,...,0.574970,2.023285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2006,3.181621,0.223387,-12028455.85,1.104362,0.505974,0.832742,2.683214,1.255184,0.066942,...,0.743588,0.416767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2007,1.095075,0.238187,3049814.24,0.129920,0.832039,1.225166,1.372838,0.941762,-0.247270,...,0.886293,1.504313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2008,2.218927,0.258117,12912489.67,0.645264,0.900856,0.831692,1.417204,1.159388,-0.265133,...,1.070836,1.070409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2009,0.701032,0.260640,42063512.92,0.990132,0.896783,0.930596,1.501864,1.241106,0.075441,...,1.387068,0.701708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2010,1.726699,0.712908,-31938852.50,0.286495,0.975603,1.236623,0.641707,0.489171,0.153819,...,2.185922,0.506915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2011,0.799126,0.379526,10172672.07,1.862770,1.241959,0.798826,0.971927,0.987812,0.123391,...,0.567295,1.683975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
